# An Introduction to Window Functions
In the first lesson, we’ll learn what window functions are, how they work, and the syntax of the OVER clause and its elements.
After we finish this lesson, we’ll be able to:
* Describe what window functions are
* Understand different types of window functions
* Define a window with the OVER() clause
* Divide a window into partitions using PARTITION BY clause


## Discovering Window Functions
Until now, we've been able to answer questions about aggregating data using the `GROUP BY` clause, but sometimes we'll encounter questions in our analysis that go beyond the capabilities of GROUP BY. Here are a few examples:  

* What's the 7-day rolling average of daily sales?
* What's the year-to-date sales growth rate compared to the same period last year?
* What's the difference between each employee’s salary and their department’s average salary?  

SQL has a solution for handling these complex questions: window functions. Throughout this course, we'll learn how to answer questions like these and see how powerful window functions can be.

While there are similarities, significant differences exist between window functions and the `GROUP BY` clause. Let’s compare and contrast the two to help us better understand window functions.

**GROUP BY**  

Adding the `GROUP BY` clause to a SQL query enables us to apply aggregate functions, such as `SUM`, `MIN`, `COUNT`, or `AVG`, to groups of rows. The `GROUP BY` clause collapses the individual rows with the same value for the columns in the `GROUP BY` clause into a single row that contains summary values.
Let’s take a look at an example to understand the concept properly. Suppose we have a table containing the capital bike share’s trip data stored in a PostgreSQL database as follows: 
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>end_date</th>
  <th>duration</th>
  <th>start_station_number</th>
  <th>start_station</th>
  <th>end_station_number</th>
  <th>end_station</th>
  <th>bike_number</th>
  <th>member_type</th>
</tr>
<tr>
  <td>2017-10-01 01:00:00.000000</td>
  <td>2017-10-01 01:04:00.000000</td>
  <td>285</td>
  <td>31400</td>
  <td>Georgia &amp; New Hampshire Ave NW</td>
  <td>31404</td>
  <td>9th &amp; Upshur St NW</td>
  <td>W22821</td>
  <td>Member</td>
</tr>
<tr>
  <td>2017-10-01 01:01:00.000000</td>
  <td>2017-10-01 01:15:00.000000</td>
  <td>816</td>
  <td>31522</td>
  <td>2nd St &amp; Seaton Pl NE</td>
  <td>31618</td>
  <td>4th &amp; East Capitol St NE</td>
  <td>W00940</td>
  <td>Member</td>
</tr>
<tr>
  <td>2017-10-01 01:01:00.000000</td>
  <td>2017-10-01 01:34:00.000000</td>
  <td>2000</td>
  <td>31044</td>
  <td>King St &amp; Patrick St</td>
  <td>31041</td>
  <td>Prince St &amp; Union St</td>
  <td>W00201</td>
  <td>Casual</td>
</tr>
<tr>
  <td>2017-10-01 01:02:00.000000</td>
  <td>2017-10-01 01:15:00.000000</td>
  <td>798</td>
  <td>31288</td>
  <td>4th St &amp; Madison Dr NW</td>
  <td>31266</td>
  <td>11th &amp; M St NW</td>
  <td>W23360</td>
  <td>Member</td>
</tr>
<tr>
  <td>2017-10-01 01:02:00.000000</td>
  <td>2017-10-01 02:09:00.000000</td>
  <td>3978</td>
  <td>31258</td>
  <td>Lincoln Memorial</td>
  <td>31233</td>
  <td>17th &amp; K St NW / Farragut Square</td>
  <td>W22595</td>
  <td>Casual</td>
</tr>
</table>
</body>
</html>



Each row represents one trip. Most fields are self-explanatory, except `member_type`, which takes two values:

* `Member`, which indicates a monthly membership
* `Casual`, which indicates that the bike rider bought a 3-day pass.

Let’s obtain the average trip’s duration for each member type in seconds.

```sql
SELECT member_type, AVG(duration) AS avg_trip_duration
  FROM tbl_bikeshare
 GROUP BY member_type;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>member_type</th>
  <th>avg_trip_duration</th>
</tr>
<tr>
  <td>Casual</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
  <td>Member</td>
  <td>733.0706225680933852</td>
</tr>
</table>
</body>
</html>


As shown above, adding the `GROUP BY` clause to the query returns a summary row for each unique grouping, not the entire result set. The result set loses the detail columns except for those in the `GROUP BY` clause.

**Window Functions**  

In contrast, queries with window functions perform aggregation without requiring a `GROUP BY` clause. Unlike queries with a `GROUP BY` clause, the individual rows are not collapsed. In other words, even though window functions enable us to apply aggregate functions like `AVG()`, `SUM()`, and `COUNT()` on a set of rows, individual rows are still accessible.

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/772-909/1.1-m772.svg">
</center>

The true power of window functions is that they can return all the columns in the `SELECT` list along with the result of the expression with the window function as just another column.

Run the following query in the code editor on the right, and check the result.


```sql
SELECT bike_number, member_type, duration,
       AVG(duration) OVER(PARTITION BY member_type) AS avg_trip_duration
  FROM tbl_bikeshare;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>bike_number</th>
  <th>member_type</th>
  <th>duration</th>
  <th>avg_trip_duration</th>
</tr>
<tr>
  <td>W00201</td>
  <td>Casual</td>
  <td>2000</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
  <td>W22595</td>
  <td>Casual</td>
  <td>3978</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
  <td>W20604</td>
  <td>Casual</td>
  <td>2259</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
  <td>W22038</td>
  <td>Casual</td>
  <td>433</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
  <td>W00035</td>
  <td>Casual</td>
  <td>1890</td>
  <td>2223.7435897435897436</td>
</tr>
<tr>
<td colspan="4" align='center'><b>. . .</b></td>
</tr>
<tr>
  <td>W22821</td>
  <td>Member</td>
  <td>285</td>
  <td>733.0706225680933852</td>
</tr>
<tr>
  <td>W00940</td>
  <td>Member</td>
  <td>816</td>
  <td>733.0706225680933852</td>
</tr>
<tr>
  <td>W23360</td>
  <td>Member</td>
  <td>798</td>
  <td>733.0706225680933852</td>
</tr>
<tr>
  <td>W20671</td>
  <td>Member</td>
  <td>1325</td>
  <td>733.0706225680933852</td>
</tr>
<tr>
  <td>W21107</td>
  <td>Member</td>
  <td>671</td>
  <td>733.0706225680933852</td>
</tr>
</table>
</body>
</html>

We can see that the result includes the individual rows and the result of the window function (in the fourth column).

We'll work on understanding the syntax of the query above in the rest of this lesson. We'll also explore, more generally, the window function syntax and its details.

As a heads up: window functions aren't limited to only aggregate functions. We'll learn different categories of functions in the following lessons.

### Instructions
1. Feel free to experiment with the second query above. For example, you might try changing the type of aggregate function or the column name after `PARTITION BY`. How does this affect the output?

## Types of Window Functions in SQL
In response to the growing demand for powerful data analysis tools in recent years, several new window functions have been introduced to SQL databases. This trend will likely continue as data analysis becomes more complex and sophisticated, and we expect to see the development of even more advanced window functions in the future. For now, we can categorize the existing window functions in SQL supported by PostgreSQL into four groups:

* **Aggregate Functions:** We're all familiar with aggregate functions like `SUM()`, `AVG()`, and `COUNT()` in queries with `GROUP BY` clause. The aggregate function must operate on a set, whether a `GROUP BY` clause or a window specification defines the set. We'll discuss them in lesson 3.

* **Ranking Functions:** Ranking and indexing data to figure out which rows are the best in the dataset can be done with ranking functions like `RANK()`, `DENSE_RANK()`, `ROW_NUMBER()`, and `NTILE()`. We'll discuss them in lesson 4.

* **Distribution Functions:** This type of window function includes `CUME_DIST()` and `PERCENT_RANK()`, which provide information on the data distribution and are used in statistical analysis. We'll discuss them in lesson 5.

* **Offset Functions:** There are two categories of window offset functions. The first category includes the `LAG()` and `LEAD()` functions, which return a value from a specific row before and after the current row. The second category includes the `FIRST_VALUE()`, `LAST_VALUE()`, and `NTH_VALUE()` functions, which respectively return a value from the first, last, and *nth* rows in the current window. We'll discuss these functions in lesson 6.

## Window Function Syntax
On this screen, we'll look at window function syntax and discuss its elements. 

As mentioned earlier, SQL window functions allow us to apply various calculations to a set of rows (also called a window) related to the current row and return a single value.

The `OVER` clause defines the specification of a window and involves three core elements:

* Partitioning definition 
* Ordering definition
* Framing definition 

However, depending on the function we call, some elements aren't required; even the `OVER` clause can be empty. 

The basic syntax of the window function call as part of a `SELECT` statement is as follows:

```sql
function_name([expression]) OVER 
(
    [partition_definition]
    [order_definition]
    [frame_definition]
) [AS column_name] 

```

Let's take a quick look at the elements of the window function syntax. SQL provides different types of window functions which need to appear before the `OVER()` clause. We'll learn the functions in detail in the following lessons. The `OVER()` clause determines which rows are included in a window before a window function computes a value for each row in the window. As shown in the syntax above, the `OVER()` clause supports the following arguments:

* The `PARTITION BY` expression divides the result set into partitions, or groups, based on one or more columns. The window function is then applied to each partition separately.

* The `ORDER BY` expression determines the order in which the window function is applied. Results can be ordered in ascending (`ASC`) or descending (`DESC`) order.

* The `frame_definition` defines a frame bounded by the `partition_definition` section, so a given window function will be applied to the rows in the frame. Framing is an advanced topic we'll cover in the next lesson.

## The OVER() Clause

Let's look deeper at the `OVER()` clause on this screen.

Before diving into the details, let's explore the `employee` table, which represents the details of a company’s employees, by running the following query in the code runner.


```sql
SELECT * 
  FROM employee;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>employee_id</th>
  <th>last_name</th>
  <th>first_name</th>
  <th>department</th>
  <th>title</th>
  <th>hire_date</th>
  <th>salary</th>
</tr>
<tr>
  <td>1</td>
  <td>Adams</td>
  <td>Andrew</td>
  <td>Management</td>
  <td>General Manager</td>
  <td>2002-08-13</td>
  <td>108000.00</td>
</tr>
<tr>
  <td>2</td>
  <td>Edwards</td>
  <td>Nancy</td>
  <td>Sales</td>
  <td>Sales Manager</td>
  <td>2002-04-30</td>
  <td>98900.00</td>
</tr>
<tr>
  <td>3</td>
  <td>Peacock</td>
  <td>Jane</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2002-03-31</td>
  <td>87000.00</td>
</tr>
<tr>
  <td>4</td>
  <td>Park</td>
  <td>Margaret</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-05-02</td>
  <td>69800.00</td>
</tr>
<tr>
  <td>5</td>
  <td>Johnson</td>
  <td>Steve</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-10-16</td>
  <td>76500.00</td>
</tr>
<tr>
  <td>6</td>
  <td>Mitchell</td>
  <td>Michael</td>
  <td>IT</td>
  <td>IT Manager</td>
  <td>2003-10-16</td>
  <td>89900.00</td>
</tr>
<tr>
  <td>7</td>
  <td>King</td>
  <td>Robert</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-01-01</td>
  <td>67800.00</td>
</tr>
<tr>
  <td>8</td>
  <td>Callahan</td>
  <td>Laura</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-03-03</td>
  <td>78000.00</td>
</tr>
<tr>
  <td>9</td>
  <td>John</td>
  <td>Edward</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-09-18</td>
  <td>75900.00</td>
</tr>
</table>
</body>
</html>

As we've seen, the data consists of the details of nine employees from different departments. 

What if we want aggregated data, such as average salary, and non-aggregated data, such as employee details?
There are different ways to implement this query. However, the most efficient approach is with window functions and the `OVER()` clause.

Let's do it and see the result. 

```sql
SELECT first_name, last_name, department, title, salary,
       AVG(salary) OVER() AS average_salary
  FROM employee;
```


<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>first_name</th>
  <th>last_name</th>
  <th>department</th>
  <th>title</th>
  <th>salary</th>
  <th>average_salary</th>
</tr>
<tr>
  <td>Andrew</td>
  <td>Adams</td>
  <td>Management</td>
  <td>General Manager</td>
  <td>108000.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Nancy</td>
  <td>Edwards</td>
  <td>Sales</td>
  <td>Sales Manager</td>
  <td>98900.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Jane</td>
  <td>Peacock</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>87000.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Margaret</td>
  <td>Park</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>69800.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Steve</td>
  <td>Johnson</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>76500.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Michael</td>
  <td>Mitchell</td>
  <td>IT</td>
  <td>IT Manager</td>
  <td>89900.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Robert</td>
  <td>King</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>67800.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>Laura</td>
  <td>Callahan</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>78000.00</td>
  <td>83533.333333333333</td>
</tr>
<tr>
  <td>John</td>
  <td>Edward</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>75900.00</td>
  <td>83533.333333333333</td>
</tr>
</table>
</body>
</html>

By specifying a blank `OVER()` clause, we can define a window that covers the whole data set. Consequently, the calculation is done across all the rows. In this example,  we get the average salary across the entire table.

In most scenarios, however, we'd like to restrict the windows using the `PARTITION BY` clause. In the next screen, we'll discuss the `PARTITION BY` clause in more detail.

Let's use the `OVER()` clause to write a query that finds the difference between employee's salaries and the company's average salary.

## Partitioning a Window
Partitioning a window divides the window into smaller windows based on one or more columns. The `PARTITION BY` expression limits the visible window to only rows with the same values as the current row. In other words, each window consists only of the rows matching the `PARTITION BY` column or columns.

Let's practice partitioning with an example.<br>
Suppose we're interested in knowing the average salary in each department.

```sql
SELECT first_name, last_name, department, title, salary,
       AVG(salary) OVER(PARTITION BY department) AS average_salary
  FROM employee;
```
<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/772-909/5.2-m772.svg">
</center>

The query above:

* Uses `AVG()` to calculate the average salary
* Partitions the data by the `department` column

Because we partitioned the data by `department`, the `salary` values from one department were not retrieved by rows of another department.


### Instructions
We're interested in knowing the details of employees and how many employees work for the same department as the current employee. 

1. Write a query that returns the following columns:

* `first_name`
* `last_name`
* `department`
* `title`
* The number of employees working for the same department as the current employee. Alias this as  `num_employees_department`.

### - Use the `COUNT()` aggregate window function before the `OVER()` clause.
* Use `PARTITION BY department` within the `OVER()` clause to return the number of employees in each department.

```sql
SELECT first_name, 
       last_name, 
       department,
       title,
       COUNT(employee_id) OVER(PARTITION BY department) AS num_employees_department
  FROM employee;
```

## Ordering a Window
The logical order of the rows within the complete result set or each partition of the result set can be defined with the `ORDER BY` clause. Simply put, the rows within each partition can be sorted based on the order specification.

The following example uses the `ORDER BY` clause to return the running total of the trip's duration for a specific bike.

```sql
SELECT start_date, bike_number, duration,
       SUM(duration) OVER (ORDER BY start_date) AS running_total
  FROM tbl_bikeshare 
 WHERE bike_number='W22661';
```

The above snippet takes the sum of multiple trips' durations over the entire result set, which is ordered by the trips' start dates.

The `ORDER BY` clause serves completely different purposes in different types of window functions, which makes it one of the most confusing elements of the `OVER()` clause. However, there's no need to worry about these challenging concepts now; we'll cover them in the following lessons.

### Instructions
Consider the table `apple_sales_quantity_by_month`, which represents the sale quantities of Apple phones in each month:

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>50</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>40</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>30</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>47</td>
</tr>
</table>
</body>
</html>



1. Write a query that returns the `sales_date`, `brand`, `quantity`, and running total of monthly sales aliased as `running_total_quantity`.

### Hint
1. Use the `SUM()` function to calculate the summation of rows in the sliding window.
2. Use the `ORDER BY` clause within the `OVER()` clause and sort the rows based on the `sales_date` column in ascending order.


## Review
Congratulations! You've completed your first lesson on window functions!

In this lesson, you learned about SQL window functions and explored the essential aspects of window functions and the `OVER()` clause.

Now you have a good understanding of the following:

* What window functions are
* The benefits of using window functions
* Different types of window functions
* The syntax of window functions and the `OVER()` clause
* How to narrow a window function's application to a specific row with a result set with `PARTITION BY` expression
* How to order a window with the `ORDER BY` expression

In the next lesson, we'll learn what a window frame is and how to define window frame bounds in SQL.

Happy learning!


## Takeaways
## Syntax

* General window functions:

  ```
  function_name([expression]) OVER 
  (
      [partition_definition]
      [order_definition]
      [frame_definition]
  ) [AS column_name] 
  ```

## Concepts

* SQL window functions allow us to apply various calculations to a set of rows (also called a window) related to the current row and return a single value.

* In general, there are four types of window functions:

* Aggregate Functions

* Ranking Functions

* Distribution Functions 

* Offset Functions

* The OVER clause defines the specification of a window and involves three core elements:

* Partitioning definition
* Ordering definition
* Framing definition

## References 

* [Window Functions - PostgreSQL](https://www.postgresql.org/docs/current/tutorial-window.html)
* [Window Functions - MySQL](https://dev.mysql.com/doc/refman/8.0/en/window-functions.html)


#DQ

# Window Function Framing
In this lesson, we’ll learn what a window frame is and how to define window frame bounds in SQL.
* Explain window framing 
* Identify framing key terms
* Create frames using the ROWS BETWEEN and RANGE BETWEEN clauses
* Indicate frame bounds using the N PRECEDING and N FOLLOWING terms
* Understand the difference between the ROWS BETWEEN and RANGE BETWEEN clauses

## An Introduction to Window Framing
In the previous lesson, we looked through the basics of window functions, the `OVER()` clause syntax, and its elements. 
We already know how to divide a window into smaller subsets using the `PARTITION BY` expression. Here's the general window function syntax we learned in the previous lesson:

```sql
function_name([expression]) OVER 
(
    [partition_definition]
    [order_definition]
    [frame_definition]
) [AS column_name] 
```

In this lesson, we'll learn about window framing, which lets us specify a window smaller than the partition. For example, with framing, we can precisely define a window that starts from the beginning of the result set and stops at the current row or define a window containing three rows before and three rows after the current row.

With frames, we can define a window containing exactly those rows in relation to the current row that should be considered when computing the results.

After you finish this lesson, you'll have a good understanding of the following:

* What the window framing is, and why it's important for the accuracy of the query's results
* Defining window frames and the difference between `ROW` and `RANGE`
* Applying window frames to running and moving aggregates

Let's dive in.

## Window Framing Syntax
Window framing restricts the rows in the window partition, which applies to aggregate window functions and three offset functions. We'll discuss them in the following lessons. 

Window framing defines two endpoints in relation to the current row within the partition. The calculation applies to the frame based on the given ordering.

The standard SQL framing specification includes a `ROWS`, `RANGE`, or `GROUPS` option.
To define a window frame, we can use the following syntaxes:

* `ROWS BETWEEN starting expression AND ending expression`

* `RANGE BETWEEN starting expression AND ending expression`

* `GROUPS BETWEEN starting expression AND ending expression`


The `starting expression` supports the following terms:


<table>
    <tr>
        <th>Framing Term</th>
        <th>Definition</th>
    </tr>
    <tr>
        <td>N PRECEDING</td>
        <td>The frame starts at N&lt;sup&gt;th&lt;/sup&gt; rows before the current row. Only supported with ROWS keyboard.</td>
    </tr>
    <tr>
        <td>UNBOUNDED PRECEDING</td>
        <td>The frame starts at the first row of the partition.</td>
    </tr>
    <tr>
        <td>CURRENT ROW</td>
        <td>The row of the current calculation.</td>
    </tr>
</table>

Also, the `ending expression` supports the following terms:

<table>
    <tr>
        <th>Framing Term</th>
        <th>Definition</th>
    </tr>
    <tr>
        <td>M FOLLOWING</td>
        <td>The frame ends at the M&lt;sup&gt;th&lt;/sup&gt; row after the current row. Only supported with ROWS keyboard.</td>
    </tr>
    <tr>
        <td>UNBOUNDED FOLLOWING</td>
        <td>The frame ends at the final row of the partition.</td>
    </tr>
    <tr>
        <td>CURRENT ROW</td>
        <td>The row of the current calculation.</td>
    </tr>
</table>

 

The figure below illustrates the key window framing terms in SQL.

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/773-909/2.1-m773.svg">
</center>

Window framing allows us to look around the neighborhood of a row. For example, we might want to calculate in a sliding window that looks at the previous, current, and subsequent value of sales quantity and average them as follows: 

```sql
SELECT *,
       AVG(quantity) OVER(ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS avg_quantity
  FROM apple_sales_quantity_by_month;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
  <th>avg_quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>50</td>
  <td>45</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>40</td>
  <td>38.33</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>25</td>
  <td>31.66</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>30</td>
  <td>34</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>47</td>
  <td>39</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>40</td>
  <td>43.5</td>
</tr>
</table>
</body>
</html>


Stay calm if you don't understand the query above; you'll get to the heart of window framing in this lesson.

In the following screens, we'll discuss the `ROWS` and `RANGE` keywords and the bound terms mentioned above and provide several examples and exercises to deepen your knowledge. 

Since most DBMSs don't support the `GROUPS` option, we won't cover it in this lesson.


## The RANGE and ROWS Operators
To better understand the window framing concept, consider the following table, which shows details of sold iPhones reported at the end of each month from January through June 2022.

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>50</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>40</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>30</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>47</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>40</td>
</tr>
</table>
</body>
</html>

Let's assume we're asked to calculate the running total quantity of sold iPhones. The query below returns the desired output:
```sql
SELECT *,
       SUM(quantity) OVER(ORDER BY sales_date
                          RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total_quantity
  FROM apple_sales_quantity_by_month;
  
```

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/773-909/3.1-m773.gif">
</center>

As shown above, to calculate the running total for every row in the partition, the sliding frame starts from the first row of the partition and ends with the current row.

Two possible frame expressions that calculate accumulating aggregate like a running total are:
```sql
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
```
and
```sql
RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
```
Later in this lesson, we'll discuss the difference between `ROWS` and `RANGE` and why we prefer to use `ROWS` instead of `RANGE` for defining frames.

In fact, for calculating a running total: the window for row 1 includes row 1; row 2 includes rows 1 and 2; and row 3 includes rows 1, 2, and 3.

### Instructions
Write a query that returns a **reverse** running total quantity of sold iPhones.

1. Create a frame within the `apple_sales_quantity_by_month` table that starts at the current row and includes all the rows to the end of the window.
2. Return all the table's columns and a new column containing the result of applying the `SUM()` function to the frame created in the previous step, aliased as `reverse_running_total_quantity`.

### Hint
1. Use `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING` to create a reverse running total.

```sql
SELECT *,
       SUM(quantity) OVER(ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS reverse_running_total_quantity
  FROM apple_sales_quantity_by_month;
```
## The RANGE and ROWS Operators - Syntax Shortcut
We also have a way to obtain the same kind of result in a shorter way:

* Without `ORDER BY` in `OVER()`, the default frame specification is:

`ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`

* With `ORDER BY` in `OVER()`, the default frame specification is:

`RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`


Therefore, just by adding an `ORDER BY` expression to an `OVER()` clause, instead of . . .

```
SELECT *,
       SUM(quantity) OVER(ORDER BY sales_date
                          RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total_quantity
  FROM apple_sales_quantity_by_month;
```
. . . we can calculate the running total as follows.

```sql
SELECT *,
       SUM(quantity) OVER(ORDER BY sales_date) AS running_total_quantity
  FROM apple_sales_quantity_by_month;
```

<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
  <th>running_total_quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>50</td>
  <td>50</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>40</td>
  <td>90</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>25</td>
  <td>115</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>30</td>
  <td>145</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>47</td>
  <td>192</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>40</td>
  <td>232</td>
</tr>
</table>

As we've seen, both queries return the same result sets and are equivalent.


### Instructions
Write a query that returns the running average quantity of phone sales.

1. Create a frame within the `apple_sales_quantity_by_month` table that starts from the beginning of the partition to the current row.
2. Return all the table's columns and a new column containing the result of applying the `AVG()` function to the frame created in the previous step, aliased as `running_avg`.

### Hint
1. Use either `ORDER BY` or `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` in the `OVER()` clause.


```sql
SELECT *,
       AVG(quantity) OVER(ORDER BY sales_date) AS running_avg
  FROM apple_sales_quantity_by_month;
```
## Framing using n PRECEDING and n FOLLOWING
On the previous screens, we learned how to calculate an accumulating aggregate by creating a frame that starts with the first row of the partition and ends with the current row, or starts at the current row and includes all rows to the end of the partition, using the following statements:


* `RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`

* `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`

Or


* `RANGE BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`

* `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`


In this screen, we'll learn how to create a frame by specifying a number of rows to include. This syntax can only be used with `ROWS` and allows us to define a frame that consists of a number of rows before, after, or before and after the current row.
Let's discover `n PRECEDING` and `n FOLLOWING` options by calculating the sum of every two preceding sales quantities.
```sql
SELECT *,
       SUM(quantity) OVER(PARTITION BY brand
                         ORDER BY sales_date
                         ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING) AS sum_two_preceding
  FROM phone_sales_quantity_by_month;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
  <th>sum_two_preceding</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>110</td>
  <td></td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>60</td>
  <td>110</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>85</td>
  <td>170</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>134</td>
  <td>145</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>90</td>
  <td>219</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>100</td>
  <td>224</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>117</td>
  <td></td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>75</td>
  <td>117</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>86</td>
  <td>192</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>124</td>
  <td>161</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>80</td>
  <td>210</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>89</td>
  <td>204</td>
</tr>
</table>
</body>
</html>

As shown, two partitions exist in the `phone_sales_quantity_by_month` table — Apple and Samsung — and the frame should consist of the two previous rows of the current row. The frame starts two rows before the current row and ends one row before the current row. Since the first row in each partition doesn't have preceding rows, the first frame is empty. So the first sum of both partitions is `NULL`. 

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/773-909/5.1-m773.svg">
</center>

The second frames include the first row of each partition and the second row's output is the first row's quantity. 

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/773-909/5.2-m773.svg">
</center>

The remaining rows in each partition return the sum of the two previous rows.



Let's explore the `n FOLLOWING` option by executing one more query that returns the average revenue for every row calculated by taking one previous row, the current row and one following row.

```sql
SELECT *,
       AVG(revenue) OVER(PARTITION BY brand
                         ORDER BY sales_date
                         ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS average_revenue
  FROM phone_sales_revenue_by_month;
```

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/773-909/5.3-m773_v2.gif">
</center>

### Instructions
Write a query that calculates the average revenue of all previous sales for the same brand.

1. Create a window and partition it by brand names.
2. Sort the partitions based on the `sales_date` column in ascending order.
3. Create a frame within the `phone_sales_revenue_by_month` table that starts at the beginning of the partition and includes all  rows before the current row.
4. Return all the table's columns and a new column containing the average revenue of rows existing in the frame created in the previous step. Alias this column as `avg_previous_revenues`.

### Hint
1. Use `ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING` to create a frame containing all prior rows before the current row.

```sql
SELECT *,
       AVG(revenue) OVER(PARTITION BY brand
                         ORDER BY sales_date
                         ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS avg_previous_revenues
  FROM phone_sales_revenue_by_month;
```

## The Difference Between ROWS and RANGE

Let's discover the `RANGE` operator by explaining the difference between `ROWS` and `RANGE` operators. 
The `RANGE` operator allows us to define a frame that includes rows with the same value in the column used in the `ORDER BY` clause.

We learned how to calculate a running total with window functions on the previous screens. Therefore, we could write the following query to calculate the running total quantity of all Samsung phones sold from different models, sorted by `sales_date`.

```sql
SELECT *,
       SUM(quantity) OVER(
                            ORDER BY sales_date
                            ROWS UNBOUNDED PRECEDING
                          ) AS cumulative_quantity_with_rows
  FROM samsung_sales_quantity;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
  <th>cumulative_quantity_with_rows</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>40</td>
  <td>40</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>30</td>
  <td>70</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>35</td>
  <td>105</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>25</td>
  <td>130</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>60</td>
  <td>190</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>25</td>
  <td>215</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>30</td>
  <td>245</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>47</td>
  <td>292</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>76</td>
  <td>368</td>
</tr>
</table>
</body>
</html>


What if instead of monitoring the changes in running total quantities of sold phones during a single month, we want to know the running total quantities of all sales in a single month? 
Add their sold phones' quantities together if there are multiple sales in a single month.

```sql
SELECT *,
       SUM(quantity) OVER(
                            ORDER BY sales_date
                            RANGE UNBOUNDED PRECEDING
                          ) AS cumulative_quantity_with_range
  FROM samsung_sales_quantity;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
  <th>cumulative_quantity_with_range</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>40</td>
  <td>70</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>30</td>
  <td>70</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>35</td>
  <td>105</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>25</td>
  <td>190</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>60</td>
  <td>190</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>25</td>
  <td>215</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>30</td>
  <td>292</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>47</td>
  <td>292</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>76</td>
  <td>368</td>
</tr>
</table>
</body>
</html>


As we've seen, rows with the same sales date share the running total quantity.

- - -
**NOTE**

The `RANGE` operator doesn't support `n PRECEDING` and `n FOLLOWING` since it is not easy to recognize **`n`** numbers preceding or following a specific value for a DBMS.
- - -
### Instructions
Write a query that returns reverse running total sales quantities for the same month or any later month.

1. To write the query, use the `samsung_sales_quantity` table.
2. Sort the window based on the `sales_date` column in ascending order.
3. Create a frame that starts at the current row and includes all the rows to the end of the window.
4. Return all the table's columns and a new column containing the total quantity of all sales from the current month and any later month. Alias this as `rev_run_quantity`.

### Hint
1. Use `RANGE BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING` to create a frame containing all the rows from the current row to the end of the window.


## Review
In this lesson, we learned the following:

* The concept and syntax of window function framing in SQL
* How to create frames using the `ROWS BETWEEN` and `RANGE BETWEEN` operators and the difference between them
* How to indicate frame bounds using the following terms: 
  * `CURRENT ROW` 
    * `UNBOUNDED PRECEDING`
    * `UNBOUNDED FOLLOWING`
    * `N PRECEDING`
    * `N FOLLOWING`

    
In the next lesson, we'll learn what aggregate window functions are and how to add aggregate window functions to non-aggregate and aggregate queries.


## Takeaways
## Syntax
* To define a window frame, we can use one of the following:


  ```
  ROWS BETWEEN starting expression AND ending expression
  RANGE BETWEEN starting expression AND ending expression
  ```


* The **starting expression** supports the following terms:

      <table>
        <tr>
            <td>**Framing Term**</td>
            <td>**Definition**</td>
        </tr>
        <tr>
            <td>`N PRECEDING`</td>
            <td>The frame starts at N&lt;sup&gt;th&lt;/sup&gt; rows before the current row. Only supported with ROWS keyword.</td>
        </tr>
        <tr>
            <td>`UNBOUNDED PRECEDING`</td>
            <td>The frame starts at the first row of the partition.</td>
        </tr>
        <tr>
            <td>`CURRENT ROW`</td>
            <td>The row of the current calculation.</td>
        </tr>
    </table>

* The **ending expression** supports the following terms:

      <table>
        <tr>
            <td>**Framing Term**</td>
            <td>**Definition**</td>
        </tr>
        <tr>
            <td>`N FOLLOWING`</td>
            <td>The frame ends at the N&lt;sup&gt;th&lt;/sup&gt; row after the current row. Only supported with ROWS keyword.</td>
        </tr>
        <tr>
            <td>`UNBOUNDED FOLLOWING`</td>
            <td>The frame ends at the final row of the partition.</td>
        </tr>
        <tr>
            <td>`CURRENT ROW`</td>
            <td>The row of the current calculation.</td>
        </tr>
    </table>



## Concepts

* Window framing allows us to define a window containing exactly those rows in relation to the current row that should be considered when computing the results.
* The `RANGE` operator allows us to define a frame that includes rows with the same value in the column used in the `ORDER BY` clause.
* The default frame specification for `ORDER BY` inside of `OVER()` is `RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`, which makes it possible to calculate running aggregates without specifying the frame when `ORDER BY` is used.

## Resources
* [Window Function Frame Specifications](https://docs.oracle.com/cd/E17952_01/mysql-8.0-en/window-functions-frames.html)


#DQ

# Window Aggregate Functions
In this lesson, we’ll learn what window aggregate functions are and how to use them to solve common queries in SQL.

* Define window aggregate functions
* Add window aggregate functions to non-aggregate queries
* Add window aggregate functions to aggregate queries

## Window Aggregate Functions

In the previous lessons, we learned the basics of window functions and one of the most challenging concepts in window functions: framing. 

In this lesson, we'll discuss window aggregate functions. Although aggregate functions and the aggregation concept are intuitive and familiar to most database users, and we used them as go-to examples in the previous lessons, this familiarity can confuse even the most experienced SQL developers when they want to learn the aggregate window functions.

Generally, when we write aggregate queries, the columns not included in the `GROUP BY` clause do not appear in the result set, and we lose the details. In the example below, we lose brand and model information for the sales items because they are only aggregated on the date.
<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/774-909/1.1-m774.gif">
</center>
The `OVER` clause allows us to add aggregate functions to non-aggregate and even aggregate queries and eliminate the mentioned restriction. Thus, a window aggregate function doesn’t require a `GROUP BY` clause. So we can implement more insightful queries without dealing with the limits of the `GROUP BY` clause.

After you finish this lesson, you'll have a good understanding of the following:

* What window aggregate functions are
* How to add window aggregate functions to non-aggregate queries
* How to add window aggregate functions to aggregate queries

Let's dive in.

## What Are Window Aggregate Functions?
Window aggregate functions are the same functions as grouped aggregate functions. One of the differences is how we apply these two classes of functions to a set of rows. When you use an aggregate function with a `GROUP BY` clause, you can compute one result from the group of rows in the result set. However, the window aggregate functions are calculated over a range of rows defined by an `OVER()` clause.

Another difference lies in the output: while grouping gives us results per group, with `OVER()`, we get results for each row.

Let's discover the difference between them with an example. 

Consider the following table:

<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>40</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>30</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>35</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>60</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>30</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>47</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>76</td>
</tr>
</table>



We wantto know the total and average quantities of phones sold each month. Let's look at the result of the query below:

```sql
SELECT sales_date, SUM(quantity), AVG(quantity)
  FROM phone_sales_quantity
 GROUP BY sales_date;
```

<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>sum</th>
  <th>avg</th>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>25</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>35</td>
  <td>35</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>76</td>
  <td>76</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>77</td>
  <td>38.5</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>85</td>
  <td>42.5</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>70</td>
  <td>35</td>
</tr>
</table>


As you've seen, with simple aggregates, the only columns allowed in the SELECT clause are the aggregate and those columns included in a GROUP BY clause, which means you no longer have access to each sale's quantity contributing to the results. So, if we add the `quantity` column in the `SELECT` clause, the DBMS yields an error message. Let's try it:

```sql 
SELECT sales_date, quantity,
       SUM(quantity),
       AVG(quantity)
  FROM phone_sales_quantity
 GROUP BY sales_date;
```

```
ERROR: column "phone_sales_quantity.quantity" must appear in the GROUP BY clause or be used in an aggregate function.
```

To handle the issue, you can use window functions to retrieve the underlying data along with the aggregates.

As we learned, the keyword for window functions is the `OVER()` clause. Here is a query that uses an `OVER` clause to fix the previous query's issue.

```sql
SELECT sales_date, brand, model, quantity,
       SUM(quantity) OVER(PARTITION BY sales_date),
       AVG(quantity) OVER(PARTITION BY sales_date)
  FROM phone_sales_quantity;
```
Unlike the query with the `GROUP BY` clause, in the preceding query, we can include additional columns in the `SELECT` clause.

The query above includes four columns of detailed data about each sale and two aggregates. The only difference between these two queries is that we used the `OVER()` clause instead of the `GROUP BY` clause in the preceding query. 

Therefore, the sum and average quantities of phones sold each month returned with detailed data as follows.

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
  <th>sum</th>
  <th>avg</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>40</td>
  <td>70</td>
  <td>35</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>30</td>
  <td>70</td>
  <td>35</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>35</td>
  <td>35</td>
  <td>35</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>25</td>
  <td>85</td>
  <td>42.5</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>60</td>
  <td>85</td>
  <td>42.5</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>25</td>
  <td>25</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>30</td>
  <td>77</td>
  <td>38.5</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Samsung Galaxy S22 Ultra</td>
  <td>47</td>
  <td>77</td>
  <td>38.5</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Samsung Galaxy Z Fold4</td>
  <td>76</td>
  <td>76</td>
  <td>76</td>
</tr>
</table>
</body>
</html>



Although having the aggregates returned for every row doesn't make sense, it allows us to write more valuable queries, like comparing aggregates with detail values, which is only simple with window functions.

```sql
SELECT *,
       AVG(quantity) OVER(PARTITION BY sales_date) AS average,
       quantity - AVG(quantity) OVER(PARTITION BY sales_date) AS difference
  FROM phone_sales_quantity;
```


## Using Window Aggregate Functions
Based on standard SQL, window aggregate functions support three main elements:

* Partitioning
* Ordering
* Framing

Here's the general syntax:

```sql
aggregate_function(column_name) OVER(
        [PARTITION BY clause],
        [ORDER BY clause],
        [window framing clause]
        )
```

Although we're already familiar with the window functions syntax and its elements, let's first review the three main elements with a focus on their usage in window aggregate functions before we introduce some helpful window aggregate functions.

* The window partitioning element allows us to restrict the window to only those rows with the same value in the partitioning columns as the current row.
* The ordering element in the window specification is not supported for window aggregate functions and gives meaning to the framing element.
* The framing element enables us to specify further which rows are included in an aggregate. So we can dive into the data section that we're interested in.


<table border="1" style="border-collapse:collapse">
    <caption>The List of Window Aggregate Functions</caption>
<tr>
  <th style="text-align:center">Window Aggregate Function</th>
  <th>Definition</th>
</tr>
<tr>
    <td><code>SUM(<i>expression</i>)</code></td>
  <td>Returns the sum of the rows from expression as viewed by the OVER clause.</td>
</tr>
<tr>
  <td><code>AVG(<i>expression</i>)<code></td>
  <td>Returns the average of expression as viewed by the OVER clause.</td>
</tr>
<tr>
  <td><code>MIN(<i>expression</i>)<code></td>
  <td>Finds the minimum value in a column given by expression as viewed by the OVER clause.</td>

</tr>
<tr>
  <td><code>MAX(<i>expression</i>)<code></td>
  <td>Returns the highest value in a column given by expression as viewed by the OVER clause. </td>
</tr>
<tr>
  <td><code>COUNT(<i>expression</i>)<code></td>
  <td>Returns a count of the number of non-NULL values of
the expression as viewed by the OVER clause.</td>
</tr>
<tr>
  <td><code>VARIANCE(<i>expression</i>)<code></td>
  <td>Returns the variance of all expression values as viewed by the OVER clause.</td>
</tr>
<tr>
  <td><code>STDDEV(<i>expression</i>)<code></td>
  <td>Returns a sample standard deviation of all expression values as viewed by the OVER clause.</td>
</tr>
</table>


Let's try some of the window aggregate functions in practice. To ensure we're able to practice all aspects of the window aggregate functions, consider the following table containing employees' details of an imaginary company:


<table border="1" style="border-collapse:collapse">
<tr>
  <th>employee_id</th>
  <th>last_name</th>
  <th>first_name</th>
  <th>department</th>
  <th>title</th>
  <th>hire_date</th>
  <th>salary</th>
</tr>
<tr>
  <td>1</td>
  <td>Adams</td>
  <td>Andrew</td>
  <td>Management</td>
  <td>General Manager</td>
  <td>2002-08-13</td>
  <td>108000.00</td>
</tr>
<tr>
  <td>2</td>
  <td>Edwards</td>
  <td>Nancy</td>
  <td>Sales</td>
  <td>Sales Manager</td>
  <td>2002-04-30</td>
  <td>98900.00</td>
</tr>
<tr>
  <td>3</td>
  <td>Peacock</td>
  <td>Jane</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2002-03-31</td>
  <td>87000.00</td>
</tr>
<tr>
  <td>4</td>
  <td>Park</td>
  <td>Margaret</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-05-02</td>
  <td>69800.00</td>
</tr>
<tr>
  <td>5</td>
  <td>Johnson</td>
  <td>Steve</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-10-16</td>
  <td>76500.00</td>
</tr>
<tr>
  <td>6</td>
  <td>Mitchell</td>
  <td>Michael</td>
  <td>IT</td>
  <td>IT Manager</td>
  <td>2003-10-16</td>
  <td>89900.00</td>
</tr>
<tr>
  <td>7</td>
  <td>King</td>
  <td>Robert</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-01-01</td>
  <td>67800.00</td>
</tr>
<tr>
  <td>8</td>
  <td>Callahan</td>
  <td>Laura</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-03-03</td>
  <td>78000.00</td>
</tr>
<tr>
  <td>9</td>
  <td>Edward</td>
  <td>John</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-09-18</td>
  <td>75900.00</td>
</tr>
</table>

 
First, let's calculate the company's annual payment to the employees. 
To answer this question, we can write the query below:

```sql
SELECT SUM(salary) as company_salary
 FROM employees;
```

<table border="1" style="border-collapse:collapse">
<tr>
  <th>company_salary</th>
</tr>
<tr>
  <td>751800</td>
</tr>
</table>

However, let's say we want to keep employee details in the output. We can use the following query:
```sql
SELECT *,
       SUM(salary) OVER() as company_salary
  FROM employees;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>employee_id</th>
  <th>last_name</th>
  <th>first_name</th>
  <th>department</th>
  <th>title</th>
  <th>hire_date</th>
  <th>salary</th>
  <th>company_salary</th>
</tr>
<tr>
  <td>1</td>
  <td>Adams</td>
  <td>Andrew</td>
  <td>Management</td>
  <td>General Manager</td>
  <td>2002-08-13</td>
  <td>108000.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>2</td>
  <td>Edwards</td>
  <td>Nancy</td>
  <td>Sales</td>
  <td>Sales Manager</td>
  <td>2002-04-30</td>
  <td>98900.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>3</td>
  <td>Peacock</td>
  <td>Jane</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2002-03-31</td>
  <td>87000.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>4</td>
  <td>Park</td>
  <td>Margaret</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-05-02</td>
  <td>69800.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>5</td>
  <td>Johnson</td>
  <td>Steve</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-10-16</td>
  <td>76500.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>6</td>
  <td>Mitchell</td>
  <td>Michael</td>
  <td>IT</td>
  <td>IT Manager</td>
  <td>2003-10-16</td>
  <td>89900.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>7</td>
  <td>King</td>
  <td>Robert</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-01-01</td>
  <td>67800.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>8</td>
  <td>Callahan</td>
  <td>Laura</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-03-03</td>
  <td>78000.00</td>
  <td>751800</td>
</tr>
<tr>
  <td>9</td>
  <td>Edward</td>
  <td>John</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-09-18</td>
  <td>75900.00</td>
  <td>751800</td>
</tr>
</table>
</body>
</html>


The preceding query returns all employee details with an additional column, `company_salary`, which will have the overall annual salary for every row.

Let's assume we're interested in each department's highest and lowest salaries. To write a query that returns the mentioned information, we require a `PARTITION BY` clause, as follows:

```sql 
SELECT last_name, department, salary,
       MAX(salary) OVER(PARTITION BY department) as max_salary,
       MIN(salary) OVER(PARTITION BY department) as min_salary
  FROM employees;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>last_name</th>
  <th>department</th>
  <th>salary</th>
  <th>max_salary</th>
  <th>min_salary</th>
</tr>
<tr>
  <td>Mitchell</td>
  <td>IT</td>
  <td>89900.00</td>
  <td>89900</td>
  <td>67800</td>
</tr>
<tr>
  <td>Edward</td>
  <td>IT</td>
  <td>75900.00</td>
  <td>89900</td>
  <td>67800</td>
</tr>
<tr>
  <td>Callahan</td>
  <td>IT</td>
  <td>78000.00</td>
  <td>89900</td>
  <td>67800</td>
</tr>
<tr>
  <td>King</td>
  <td>IT</td>
  <td>67800.00</td>
  <td>89900</td>
  <td>67800</td>
</tr>
<tr>
  <td>Adams</td>
  <td>Management</td>
  <td>108000.00</td>
  <td>108000</td>
  <td>108000</td>
</tr>
<tr>
  <td>Johnson</td>
  <td>Sales</td>
  <td>76500.00</td>
  <td>98900</td>
  <td>69800</td>
</tr>
<tr>
  <td>Park</td>
  <td>Sales</td>
  <td>69800.00</td>
  <td>98900</td>
  <td>69800</td>
</tr>
<tr>
  <td>Peacock</td>
  <td>Sales</td>
  <td>87000.00</td>
  <td>98900</td>
  <td>69800</td>
</tr>
<tr>
  <td>Edwards</td>
  <td>Sales</td>
  <td>98900.00</td>
  <td>98900</td>
  <td>69800</td>
</tr>
</table>
</body>
</html>

### Instructions
A table called `phone_sales_by_month` contains sales data for two different phone brands. You want to analyze the revenue generated by each brand and view it in different ways to identify trends more accurately.

1. Write a query that returns the following:
   * All original columns in the table
   * The running average sales revenue for each phone brand separately, aliased as `running_average`
   * The rolling three-month average sales revenue for each phone brand separately (use the previous two months and the current month), aliased as `three_month_average`

    
*Note that revenue is calculated by multiplying `quantity` by `unit_price`.*

### Hint
* Use `PARTITION BY` to divide the data into two partitions for the two existing brands in the data set.
* Use `ORDER BY` to sort the data in each partition; remember that we always have to have an `ORDER BY` clause if we use the `ROWS` clause.
* You'll need two window functions in your `SELECT` statement, one for each averaging type.

```sql
SELECT *,
       AVG(quantity*unit_price) OVER(PARTITION BY brand
                        ORDER BY sales_date
                        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
           ) AS running_average,
      AVG(quantity*unit_price) OVER(PARTITION BY brand
                        ORDER BY sales_date
                        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
           ) AS three_month_average
  FROM phone_sales_by_month;
```


## Window Functions and The WHERE Clause
Here we'll learn one of the important issues related to window functions if we don't apply them to a set of rows returned by a query correctly. Let's discuss the issue by adding a `WHERE` clause to a query with an `OVER()` clause.

We're able to add a `WHERE` clause to any query for filtering the result, as follows:

```sql
SELECT first_name, last_name, department, salary,
       AVG(salary) OVER() as average_salary
  FROM employees
 WHERE department = 'Sales';
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>first_name</th>
  <th>last_name</th>
  <th>department</th>
  <th>salary</th>
  <th>average_salary</th>
</tr>
<tr>
  <td>Nancy</td>
  <td>Edwards</td>
  <td>Sales</td>
  <td>98900.00</td>
  <td>83050</td>
</tr>
<tr>
  <td>Jane</td>
  <td>Peacock</td>
  <td>Sales</td>
  <td>87000.00</td>
  <td>83050</td>
</tr>
<tr>
  <td>Margaret</td>
  <td>Park</td>
  <td>Sales</td>
  <td>69800.00</td>
  <td>83050</td>
</tr>
<tr>
  <td>Steve</td>
  <td>Johnson</td>
  <td>Sales</td>
  <td>76500.00</td>
  <td>83050</td>
</tr>
</table>
</body>
</html>

As shown above, we calculated the salaries only for the sales department since the `AVG()` function is only applied to the rows returned by the query. 

That's precisely the rule we need to remember. Window functions are always executed after the `WHERE` clause, so they're applied to whatever the result set contains.

Although we can run the above query, we cannot put window functions in a `WHERE` clause. Let's look at the following query that tries to return those employees whose salaries are higher than the average salary.

```sql
SELECT first_name, last_name, department, salary,
       AVG(salary) OVER() as average_salary
  FROM employees
 WHERE salary > AVG(salary) OVER();
```

Running the above query yields the error message:

```
ERROR: window functions are not allowed in WHERE
```

We got the above error message because if we put window functions in a `WHERE` clause, we get into a circular dependency trap. 

To solve this problem, we can employ a subquery. Let's try this!

### Instructions
1. Write a query against the `employees` table that finds `first_name`, `last_name`, and `salary` for those employees whose salaries are above the average.

* Create a subquery that returns every employee's first name, last name, and salary, along with the company's average salary.

* Query the result set returned by the subquery and select the `first_name`, `last_name`, and `salary` columns, then filter out only those employees whose salaries are above the average.

### Hint
* Create a subquery in the `FROM` clause against the `employees` table with the below columns and window function:
  * `first_name`
  * `last_name` 
  * `salary`
  * `AVG(salary) OVER() AS average_salary`


```sql
SELECT first_name, last_name, salary
  FROM (SELECT first_name, last_name, salary,
               AVG(salary) OVER() AS average_salary
          FROM employees) AS subquery
 WHERE salary > average_salary;
```

## Window Functions and the Order of Execution
We need to know SQL's logical query evaluation to understand the root of our issue when using the window functions in the `WHERE` clause.

Let's discover the query processing order since it is one of the most fundamental aspects of SQL that every SQL developer needs to learn, especially when dealing with window functions.

Always keep in mind the following order:

1. `FROM` and `JOIN`: The `FROM` clause constructs the data set for the query. Multiple data sources can be joined using the `JOIN` clause if necessary.
2. `WHERE`: The conditions defined in the `WHERE` clause determine which rows are included in the data set.
3. `GROUP BY`: The `GROUP BY` clause groups the data set's rows based on the grouping expression. 
4. `HAVING`: The `HAVING` clause filters aggregate results.
5. `SELECT`: The `SELECT` clause determines those columns appearing in the result set.
   * This is where we've been using window functions.
6. `ORDER BY`:  The `ORDER BY` clause sorts the result set based on one or multiple columns.
7. `OFFSET`: The `OFFSET` clause determines where to start returning data.
8. `LIMIT`: The `LIMIT` clause limits the number of rows returned by a `SELECT` statement.

Although we can put aggregate functions in a `HAVING` clause, given the above, it's impossible to put window aggregate functions in the `WHERE`, `GROUP BY`, and `HAVING` clauses because while these clauses are being processed, we haven't gotten to `SELECT` yet. Thus the results of the window functions still aren't ready for consumption.

### Instructions
1. Write a query against the `employees` table that returns the `first_name`, `last_name`, `department`, and `salary` columns along with the average salary of the department they are associated with, aliased as `dep_avg_salary`.

* Sort the result set based on the department's average salary in descending order.

### Hint

* Add the following clause to the query:

```sql
AVG(salary) OVER(PARTITION BY department)
```

```sql
SELECT first_name, last_name, department, salary,
       AVG(salary) OVER(PARTITION BY department) AS dep_avg_salary
  FROM employees
 ORDER BY dep_avg_salary DESC;
```

## Combining Window Aggregate Functions with Aggregate Queries
Combining window aggregates with aggregate queries is more complex than it seems. Let's look deeper into it. 

Let's assume we're asked to report the total salary for each department and company's total salary based on the data in the `employees` table. 

To respond to the question, we can try the following query:

```sql
SELECT department, SUM(salary) AS total_depatment_salary,
       SUM(salary) OVER() AS total_company_salary
  FROM employees
 GROUP BY department; 

```
Yes, it shows the error:

```
ERROR: column "employees.salary" must appear in the GROUP BY clause or be used in an aggregate function
```


To solve the issue above, we should keep in mind these two rules:

1. The result of the `FROM`, `WHERE`, `GROUP BY`, and `HAVING` clauses determines the rows and columns in the base window and the granularity.

2. Any expression used anywhere in the window function must be in the `GROUP BY` clause or aggregated inside the window function.

Based on the mentioned rules, the rows in the window are created after the `GROUP BY` operation; therefore, the window contains any expression listed in the `GROUP BY` clause plus any aggregate expression. Any expression in any part of window functions must follow the same rules as the `SELECT` list.

The above explanation means that since the `department` column is part of the `GROUP BY` clause, we can use the column as the argument of any window functions or as a `PARTITION BY` column. Also, to use the `salary` column in any window functions, it must be aggregated first.

Now, let's see the solution.

```sql
SELECT department, SUM(salary) as total_depatment_salary,
       SUM(SUM(salary)) OVER() as total_company_salary
FROM employees
GROUP BY department;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>department</th>
  <th>total_depatment_salary</th>
  <th>total_company_salary</th>
</tr>
<tr>
  <td>Management</td>
  <td>108000</td>
  <td>751800</td>
</tr>
<tr>
  <td>Sales</td>
  <td>332200</td>
  <td>751800</td>
</tr>
<tr>
  <td>IT</td>
  <td>311600</td>
  <td>751800</td>
</tr>
</table>
</body>
</html>

Although we cannot use nested aggregate functions in SQL, as mentioned earlier, the window function must be applied to the sum of `salary`, not just `salary`.

### Instructions
1. Write a query against the `phone_sales_by_month` table that returns:
   * month (extracted from `sales_date`)
   * brand
   * monthly revenue for each brand (aliased as `monthly_brand_revenue`)
   * total revenue for each brand (aliased as `total_brand_revenue`)
2. Group your results by month and brand.
3. Order your results by brand, then month.

### Hint
* Calculate the revenue by multiplying the phone quantity sold by the phone's unit price, `quantity*unit_price`. 
* Extract a month number from a date/time value with the [`EXTRACT()`](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function in PostgreSQL. 
* Use the `GROUP BY` clause with the expression `EXTRACT(MONTH from sales_date)` and the column `brand`.
* Use a regular aggregate expression to return monthly revenue for each phone's brand.
* Use a window aggregate with a `PARTITION BY` expression on the `brand` column to return the total revenue for each phone's brand.
* Sort the result set based on the brand name and month number.

```sql
SELECT EXTRACT(MONTH FROM sales_date) AS month, 
       brand, 
       SUM(quantity*unit_price) AS monthly_brand_revenue,
       SUM(SUM(quantity*unit_price)) OVER(PARTITION BY brand) AS total_brand_revenue
  FROM phone_sales_by_month
 GROUP BY EXTRACT(MONTH FROM sales_date), brand
 ORDER BY brand, EXTRACT(MONTH FROM sales_date);
```


## Review
Congratulations! You've completed lesson three of the window functions course.

In this lesson, we learned about window aggregate functions and deepened our knowledge by using them to solve common queries.

Now you have a good understanding of the following:

* What window aggregate functions are
* How to add window aggregate functions to non-aggregate queries
* How to add window aggregate functions to aggregate queries
* The logical query processing order and why window functions are restricted to only specific clauses

In the next lesson, we'll learn ranking window functions to determine the rank of a value in a group of values.

## Takeaways
## Syntax
* The general syntax for applying a window aggregate function to a window is as follows:

  ```
  aggregate_function(column_name) OVER(
          [PARTITION BY clause],
          [ORDER BY clause],
          [window framing clause]
          )
  ```


## Concepts
* Although window aggregate functions are the same as grouped aggregate functions, there are two main differences:
  * The window aggregate functions are calculated over a range of rows defined by an `OVER()` clause instead of a `GROUP BY` clause.

* Grouping gives us results per group; with `OVER()`, we get results for each row.


* We can't use window functions in the `WHERE`, `GROUP BY`, and `HAVING` clauses. 

* We're able to combine window aggregates with aggregate queries.


## Resources

* [Window Functions](https://www.postgresql.org/docs/current/tutorial-window.html)


* [Functions and Operators](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT)


#DQ

# Ranking Window Functions
This lesson provides an introduction to the `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()`, and `NTILE()` functions to generate ranking within a window. It also provides a practical, hands-on approach to using the ranking window function to solve complex analytical problems.
* Explain the purpose and benefits of using ranking window functions in SQL
* Describe the syntax and parameters of each ranking function
* Understand the differences between ranking window functions to choose the appropriate ranking function
* Use ranking window functions to solve complex problems

## Introduction to Ranking Window Functions
SQL window functions have evolved the ability to gain insights from the underlying data, and undoubtedly, ranking window functions play a significant role among them. Ranking window functions rank rows within a specified partition of a result set. 

In this lesson, we'll learn about ranking window functions and how to use them in SQL.
Ranking functions are nondeterministic functions gathered under the window functions umbrella and designed to solve complicated problems quickly.

For example, ranking window functions enable us to answer which observation is best, second best, or worst, which is hard to implement this kind of queries with conventional SQL queries.

In this lesson, we're going to learn the following four ranking functions:

* `ROW_NUMBER()`
* `RANK()`
* `DENSE_RANK()`
* `NTILE()`

The first three functions assign a sequential number to each row in a query’s result set. The last function creates a specified number of buckets (or groups) and gives each row in a query’s result set to one of these buckets according to their rank, starting with one and continuing to the number of buckets specified. 

This lesson will discuss these ranking functions in detail. After we finish this lesson, we'll have a good understanding of

* How to use the ranking window functions to create a basis for solving analytics problems.
* The difference between the `ROW_NUMBER()`, `RANK()`, and `DENSE_RANK()` functions.
* How to define a specified number of buckets with the `NTILE()` function to solve particular analytical problems.

Let’s dive in.

## Ranking Functions and Supported Windowing Elements
Before explaining the four ranking window functions in detail and delving further into the hands-on practice, let’s review which windowing elements are mandatory or optional when calling the ranking functions.

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/775-909/2.1-m775.svg">
</center>

In general, all four ranking functions support the following:

* An optional `PARTITION BY` subclause. The query's result set is considered a single partition if we don't specify a window partition clause.
* An `ORDER BY` subclause, which is not mandatory but strongly encouraged for meaningful results.

Window framing is irrelevant to the ranking window functions. The functions are always applied to the entire partition, as ranking is always relative to other rows within the same partition.

Among these three windowing elements, the `ORDER BY` subclause is much more important when using the ranking window functions. Unlike aggregate window functions, where the `ORDER BY` subclause is only used for framing purposes, in the ranking window functions, this strongly-encouraged subclause is used for determining the rank and giving meaning to the calculation.

So the syntax of the ranking functions in the simplest form is as follows:  

```sql
RANKING_FUNCTION() OVER(
						ORDER BY column_name
					   )
```

The `RANK()`, `DENSE_RANK()`, and `ROW_NUMBER()`functions do not take arguments. However, empty parentheses are required.

## The ROW_NUMBER() Function
Let’s learn the ranking window function with the most commonly used window function for data analysis, `ROW_NUMBER()`. The `ROW_NUMBER()` function returns a unique integer starting with one for each row within the respective window partition, sorted based on the specified window ordering expression.

The syntax of the ranking function is as follows:
```sql
ROW_NUMBER() OVER (
    [PARTITION BY expression,...]
    ORDER BY expression,...  
)
```

As mentioned in the previous screen, the window partitioning element is optional, and the window ordering element is necessary for meaningful, reproducible results. So, remember that we have to always include the `ORDER BY` subclause inside the `OVER()` clause when we call the ranking window functions. 


Consider the modified version of the capital bike share’s trip dataset called `trips`.
The `trips` table’s rows represent twelve bike trips with all the associated information we’ve seen in lesson one, plus the `rider_rating` column, which indicates how satisfied the rider is with riding the bike.
To understand how the `ROW_NUMBER()` function works, let’s apply it to the `trips` table and see the result.
```sql
 SELECT start_date, bike_number, member_type, rider_rating,
       ROW_NUMBER() OVER(ORDER BY rider_rating)
  FROM trips;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>row_number</th>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>7</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>8</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>9</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>10</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>11</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>12</td>
</tr>
</table>
</body>
</html>


We can also use an optional `PARTITION BY` subclause that divides the window into smaller windows. The following query uses the `PARTITION BY` subclause to divide the data into two partitions based on the `member_type` column.

```sql
SELECT start_date, bike_number, member_type, rider_rating,
       ROW_NUMBER() OVER(PARTITION BY member_type
                         ORDER BY rider_rating)
  FROM trips; 
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>row_number</th>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>7</td>
</tr>
</table>
</body>
</html>


As shown above, the row number for each partition begins with 1 and increments by 1.

Simple, isn’t it? But there's an issue: the `ROW_NUMBER` window function is nondeterministic. In other words, it may return rows in any order.

The `ROW_NUMBER` function sorts the rows in each partition, and based on the position of each row, assigns unique numbers to them. When the `ORDER BY` subclause is not unique, there's no guarantee of getting the same row number every time we run the query.

To ensure repeatable results when using the `ROW_NUMBER` function, the `ORDER BY` columns in the `OVER` clause must be unique.

Determinism and non-determinism of the `ROW_NUMBER()` function's calculation depend on the uniqueness of the window ordering clause.

### Instructions
Write a query against the `trips` table that returns the `start_date`, `bike_number`,  `member_type`, and `rider_rating` columns along with their row numbers based on their start date.

1. Write the SELECT statement to retrieve the `start_date`, `bike_number`, `member_type`, and `rider_rating` columns from the `trips` table.
2. Add the `ROW_NUMBER()` function to the SELECT statement to generate row numbers for each row based on the order of the `start_date` column.
   * Use the `OVER` clause to specify the order of the rows by the `start_date` column.

### Hint
* Use the following expression in the `SELECT` statement:

`ROW_NUMBER() OVER(ORDER BY start_date)`

```sql
SELECT start_date, bike_number, member_type, rider_rating,
       ROW_NUMBER() OVER(ORDER BY start_date)
  FROM trips;
```

## The RANK() Function
The next ranking window function we'll learn is `RANK`. It's an analytical function that returns the rank of each row within the partition of a result set. 

The syntax for the `RANK()` function is:
```sql
RANK() OVER (
  [PARTITION BY expression,...]
  ORDER BY expression,...
)
```

`ROW_NUMBER()` and `RANK()` have similar results, but differ in dealing with duplicate values. For instance, if multiple bike riders rate their riding experience the same way, `ROW_NUMBER()` would give each rider a unique number, while `RANK()` would assign the same rank to them. 

Let’s see how the `RANK()` function works by looking into the following query.
 
```sql
SELECT start_date, bike_number, member_type, rider_rating,
       RANK() OVER(ORDER BY rider_rating)
  FROM trips;
```

The first four columns are added to return more detail for each trip, but the important part of the query is calculating the rank of each row when we sort the data based on the `rider_rating` column. If we run the query, we'll see ties in the result. 

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>rank</th>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>6</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>10</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>10</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>10</td>
</tr>
</table>
</body>
</html>


Therefore, the difference between the `ROW_NUMBER()` and `RANK()` functions is that the ranking of the rows with ties will be the same when we use the latter.

Take a look at the data. Imagine our task is to identify top-rated rides. Sorting rider ratings in ascending order is unsuitable, so we'll adjust the query to sort rider ratings in descending order. This approach efficiently retrieves the highest-rated rides, allowing the company to recognize them and enhance services accordingly. 

Here’s the query:

```sql
SELECT start_date, bike_number, member_type, rider_rating,
       RANK() OVER(ORDER BY rider_rating DESC)
  FROM trips;
```
<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/775-909/4.1-m775.svg">
</center>
To better understand how the `RANK()` function works, look at the result of the above query.  The rank of each trip is given in the last column.

There are three trips with the highest rating, 5, and they all are ranked 1. But surprisingly, the fourth row with a rating of 4 is ranked 4, not 2. 

Since there are three trips with the same rating values before the fourth row, even though the first three rows are all ranked one, the rank stays the same until the value of `rider_rating` changes and the rank becomes the row number.

The `RANK()` function assigns the same rank to the rows with equal values in the `ORDER BY` column. Also, the next rank is calculated by adding the number of tied rows to the tied rank; thus, the ranks might not be consecutive.  

The illustration above explains how the RANK() function operates and highlights the difference between it and the ROW_NUMBER() function. Note that the numbers enclosed in red squares represent row numbers.

Let’s include `PARTITION BY` in our next query to discover each trip’s rank when the rows are divided into two partitions based on the available member types. 

```sql
SELECT start_date, bike_number, member_type, rider_rating,
       RANK() OVER(
           PARTITION BY member_type
           ORDER BY rider_rating DESC)
  FROM trips;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>rank</th>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>7</td>
</tr>
</table>
</body>
</html>


Looking at the results, we can see that the rankings are reset to 1 once the value of `member_type` changes.

### Instructions
Write a query against the `trips` table that ranks each **day’s** trips based on their durations. 

1. Select all the table’s columns.
2. Use the `RANK()` function to determine the rank of each trip.
   * Use the `EXTRACT()` function to extract day from the `start_date` column.
   * Sort the `duration` column so that the longest trip receives a rank of 1.

### Hint
* To create a partition for each day, use the following expression in the `OVER()` clause:

`PARTITION BY EXTRACT(DAY FROM start_date)`
* To sort the duration of trips in descending order, use the following expression in the `OVER()` clause:

`ORDER BY duration DESC`

```sql
SELECT *,
       RANK() OVER(
	   		PARTITION BY EXTRACT(DAY FROM start_date)
            ORDER BY duration DESC)
 FROM trips;
```

## The DENSE_RANK() Function
This section will teach the other helpful ranking function, `DENSE_RANK()`. It returns the same ranking for the identical values of a column specified in the `ORDER BY` subclause.

Unlike the `RANK()` function, the `DENSE_RANK()` function doesn’t leave gaps in numbering when more than one identical value is in the `ORDER BY` column. 

In other words, by changing the value of the `ORDER BY` column, the `RANK()` function returns how many rows are before the current row, and the `DENSE_RANK()` returns how many unique values are before the current value.

The syntax for the `DENSE_RANK()` function is:

```sql
DENSE_RANK() OVER (
 [PARTITION BY expression,...]
 ORDER BY expression,...
)
```

Let’s discover the `DENSE_RANK()` function by ranking trips based on the `rider_rating` column.

```sql
SELECT start_date, bike_number, member_type, rider_rating,
       DENSE_RANK() OVER(
                    ORDER BY rider_rating DESC)
  FROM trips;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>dense_rank</th>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>4</td>
</tr>
</table>
</body>
</html>


As we’ve seen, the first three rows receive a ranking of 1, and the fourth row receives a ranking of 2. The `DENSE_RANK()` function returns dense ranks, meaning numbering gaps occur. 

### Instructions
Assume your manager wants you to identify the longest bike trips for each day. Write a query against the `trips` table. 

1. Select all the table’s columns.
2. Use the `DENSE_RANK()` function to rank each day’s trips based on duration.
   * Use the `EXTRACT()` function to extract day from the `start_date` column.
   * Sort the `duration` column so the longest trip receives a rank of 1.
   * Alias your window function as `trip_dense_rank`.
5. Filter out the rows with dense rank 1. Consider using a CTE to make filtering rows with a dense rank of 1 more straightforward.

### Hint

* Write a CTE to return all the table's columns and the dense rank of each trip.
* Use the following expression in the `OVER()` clause to create a partition for each day:

`PARTITION BY EXTRACT(DAY FROM start_date)`
* Use the following expression in the `OVER()` clause to sort the duration of trips in descending order: 

`ORDER BY duration DESC`

```sql
WITH dr_query AS
         (SELECT *,
                 DENSE_RANK()
                 OVER (PARTITION BY EXTRACT(DAY FROM start_date)
                     ORDER BY duration DESC) AS trip_dense_rank
          FROM trips
          )
SELECT * 
  FROM dr_query
 WHERE trip_dense_rank = 1;
```


## Comparing The Three Ranking Window Functions
Let's deepen our understanding of the three ranking window functions by trying the following query.
The query below returns the values of the three ranking functions we've learned so far, so we can compare how they rank values.

```sql
SELECT start_date,bike_number, member_type, rider_rating,
       ROW_NUMBER() OVER(ORDER BY rider_rating DESC),
       RANK() OVER(ORDER BY rider_rating DESC),
       DENSE_RANK() OVER(ORDER BY rider_rating DESC)
  FROM trips;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>row_number</th>
  <th>rank</th>
  <th>dense_rank</th>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
  <td>1</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
  <td>2</td>
  <td>1</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>3</td>
  <td>1</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>4</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
  <td>5</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
  <td>6</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>7</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
  <td>8</td>
  <td>8</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
  <td>9</td>
  <td>8</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
  <td>10</td>
  <td>8</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
  <td>11</td>
  <td>8</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>12</td>
  <td>12</td>
  <td>4</td>
</tr>
</table>
</body>
</html>


Notice that the `rider_rating` column is not unique, which affects the output of our three ranking functions differently, and the ordering direction is descending. The result set above shows that the row numbers are unique. However, the rank and dense rank values are not unique.

All the rows with the same rider rating get the same rank and dense rank. 

For example, for all rows with rider rating 4, the rank and dense rank values are the same, 4 and 2, respectively. Rank 4 means there are three rows with higher rider ratings in the result set, and dense rank 2 means one greater distinct rider rating exists in the result set.

Use this coding screen to test out the ranking values of different fields. For example, what happens when you order by member_type or day (remember, you can use `EXTRACT` to reference the day)? Experiment until you are ready to move on.


## The NTILE() Function
The last ranking function we'll discuss is `NTILE()`. The `NTILE()` function is mainly used for analytical purposes and divides ordered rows into roughly equally-sized buckets based on a specified window ordering.

The `NTILE()` can be used to segment customers or products into different groups based on a specific attribute or behavior. 
For example, if we have a dataset of customers and their purchase history, we can use `NTILE()` to divide them into segments based on the total amount they have spent.

The syntax for the `NTILE()` function is:

```sql
NTILE(ntile_expression) 
OVER(
	[PARTITION BY expression,...]
	ORDER BY expression,...
)
```

The `NTILE()` function returns an integer indicating a specific row’s bucket number. The `ntile_expression` part specifies the number of buckets, and the numbering starts at 1.  
The size of each bucket created by the `NTILE()` function is as equal to the other buckets as possible.



The `NTILE()` function requires the `ORDER BY` subclause, and the `PARTITION BY` subclause is optional.

Let’s see how the `NTILE()` ranking function works. 
Suppose we are asked to create five buckets and assign each trip based on duration to one of them.

```sql
SELECT start_date,bike_number, duration, rider_rating,
       NTILE(5) OVER(ORDER BY duration DESC)
  FROM trips;
```

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/775-909/7.1-m775.svg">
</center>

We've created five buckets based on the `duration` column's values in the query above. In other words, the `NTILE()` function distributes the trips into five buckets. Since the rows are sorted in descending order, the longest trips have been put in bucket 1 and the shortest in bucket 5. 

The size of each bucket created by the `NTILE()` function is as equal to the other buckets as possible. That means if the number of rows is not divisible by the number of buckets specified in the `NTILE()` function, some buckets get one more row than others. 

Notice that buckets 1 and 2 — the larger ones — have an extra row and come first. 

### Instructions
The bike-sharing company prioritizes maintenance for bikes with the highest daily rider ratings to enhance customer satisfaction. Your task is to identify these top-rated bikes by writing a query that divides each day's trips into two buckets based on rider ratings — placing the highest-rated bikes in the first bucket and the lowest-rated bikes in the second bucket. This data will help the company maintain its best-performing bikes in optimal condition.


1. Select the `start_date`, `bike_number`, and `rider_rating` columns from the `trips` table.
2. Use the `NTILE()` function to divide each day’s trip into two buckets based on rider ratings.
3. Use the `EXTRACT()` function to extract day from the `start_date` column.
4. Order the data by `rider_rating` in descending order within the `OVER()` clause to place higher-rated trips in the first bucket and lower-rated trips in the second bucket.

### Hint
* To create a partition for each day, use the following expression in the `OVER()` clause:

`PARTITION BY EXTRACT(DAY from start_date)`.
* To sort the rating of trips in descending order, use the following expression in the `OVER()` clause:

`ORDER BY rider_rating DESC`.

```sql
SELECT start_date, bike_number, rider_rating,
       NTILE(2) OVER(PARTITION BY EXTRACT(DAY FROM start_date)
           ORDER BY rider_rating DESC)
  FROM trips;
```

## Solving Real-World Problems with Window Ranking Functions
In this section, we’ll learn three applications of the ranking functions for solving real-world problems.

### First Scenario
Suppose we're curious about the longest trips of every day. Using traditional SQL statements to answer this question is possible, but not as easy as the solution we’re discussing right now. 
```sql
 WITH longtrips AS
    (
        SELECT *,
        ROW_NUMBER() OVER(PARTITION BY EXTRACT(DAY FROM start_date)
         ORDER BY duration DESC) AS RowNumber
          FROM trips
    )
SELECT *
  FROM longtrips
 WHERE RowNumber = 1;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>end_date</th>
  <th>duration</th>
  <th>start_station_number</th>
  <th>start_station</th>
  <th>end_station_number</th>
  <th>end_station</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
  <th>rownumber</th>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>2017-10-01 05:26:00.000000</td>
  <td>1476</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31010</td>
  <td>S Glebe &amp; Potomac Ave</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>2017-10-02 03:47:00.000000</td>
  <td>987</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31249</td>
  <td>Jefferson Memorial</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>2017-10-03 12:23:00.000000</td>
  <td>1390</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31247</td>
  <td>Jefferson Dr &amp; 14th St SW</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>2017-10-04 08:45:00.000000</td>
  <td>918</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31235</td>
  <td>19th St &amp; Constitution Ave NW</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
  <td>1</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>2017-10-05 08:33:00.000000</td>
  <td>1482</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31633</td>
  <td>Independence Ave &amp; L&#39;Enfant Plaza SW/DOE</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
  <td>1</td>
</tr>
</table>
</body>
</html>


We've solved the problem using a CTE containing all the columns plus a row number. The row number is partitioned by day, so the numbers start over for each day. The outer query retrieves the data from the CTE and filters on the row number.

### Second Senario
The second scenario highlights applying the `DENSE_RANK()` function to return each day’s second-most-rated trip.
```sql
WITH rating AS
    (
        SELECT start_date, duration, bike_number, rider_rating,
        DENSE_RANK() OVER(PARTITION BY EXTRACT(DAY FROM start_date)
        ORDER BY rider_rating DESC) AS rank
        FROM trips
    )
SELECT *
  FROM rating
 WHERE rank = 2;
```
 
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>duration</th>
  <th>bike_number</th>
  <th>rider_rating</th>
  <th>rank</th>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>1476</td>
  <td>W23254</td>
  <td>2</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>797</td>
  <td>W20095</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>918</td>
  <td>W23268</td>
  <td>4</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>202</td>
  <td>W20184</td>
  <td>3</td>
  <td>2</td>
</tr>
</table>
</body>
</html>


The query above consists of two parts. The CTE contains the `start_date`, `duration`, `bike_number`, and `rider_rating` columns, plus the rank of each trip, which is calculated by the `dense_rank()` function. The trips are partitioned by day, so the ranks start over for each day. Finally, the outer query returns trips with rank 2.


Let's help the bike-sharing company distribute bonus money to its riders based on their ride duration. The company's management has defined four bonus pools, with riders in the first pool receiving a USD 5 bonus and riders in the last pool receiving a USD 2 bonus. 

Challenge your knowledge with a hands-on exercise.

### Instructions
Write a SQL query against the `trips` table to calculate the bonus for each rider and display the results.

1. Select the `start_date`, `bike_number`, `member_type`, and `duration` columns.
2. Use the `NTILE()` window function to divide the riders into four groups based on their ride duration. Order the data by `duration` in the `OVER()` clause.
3. Add 1 to the result of the `NTILE()` function to calculate the bonus for each rider, with the first group receiving a USD 5 bonus and the last group receiving a USD 2 bonus. Alias this column as `bonus`.
4. Order the final result set by the bonus amount in descending order to display the riders with the largest bonus at the top.

### Hint
* Remember that the `NTILE()` function takes an integer as an argument, determining the number of groups you want to divide the data into.
* Specify the appropriate ordering of the data based on the ride duration.

```sql
SELECT start_date, bike_number, member_type, duration,
       NTILE(4) OVER(ORDER BY duration) + 1 AS bonus
  FROM trips
 ORDER BY bonus DESC;
```

## Review
Congratulations! You've completed the ranking window functions lesson!


In this lesson, we learned the four ranking window functions in SQL:

* `ROW_NUMBER()`  is the simplest of the ranking window functions. It assigns a unique number to each row within a specified partition of a result set.

* `RANK()` assigns a ranking to each row within a result set based on the values of a specified column or expression, indicating its rank relative to the other rows in the result set. If two or more rows have the same value for the ranking column or expression, they'll be assigned the same rank value, and the next rank value will be skipped.

* `DENSE_RANK()` is similar to the `RANK()` function. Unlike the `RANK()` function, `DENSE_RANK()` does not skip rank values when there are ties in the data. This means that if two or more rows have the same value for the ranking column or expression, they'll be assigned the same rank value, and the next rank value will be assigned to the subsequent row(s).


* `NTILE()`  divides a result set into a specified number of equal groups, buckets, or tiles based on a specified column or expression. The `NTILE()` function assigns a unique integer value to each row, indicating which tile the row belongs to.

We discussed how the above functions work, and their difference has been highlighted. 
We also learned how we could use these functions for solving analytical problems.
In the next lesson, we'll learn about distribution window functions.

Keep it up!
## Takeaways
## Syntax

* **ROW_NUMBER()**: Assigns a unique sequential integer to each row within the result set, starting from 1 for the first row.

  ```
  ROW_NUMBER() OVER (
    [PARTITION BY col(s) ]
    ORDER BY col(s) [ASC | DESC], ...
  )
  ```

* **RANK()**: Assigns a unique rank to each row within the result set, with the same rank assigned to rows with equal values, leaving gaps between ranks for non-unique values.

  ```
  RANK() OVER (
    [PARTITION BY col(s) ]
    ORDER BY col(s) [ASC | DESC], ...
  )
  ```

* **DENSE_RANK()**: Assigns a unique rank to each row within the result set, with the same rank assigned to rows with equal values but without gaps between ranks for non-unique values.



  ```
  DENSE_RANK() OVER (
    [PARTITION BY col(s) ]
    ORDER BY col(s) [ASC | DESC], ...
  )
  ```

* **NTILE(n)**: Divides the result set into a specified number (n) of approximately equal-sized groups, assigning each row to a group with a unique integer identifier ranging from 1 to n.

  ```
  NTILE(n) OVER (
    [PARTITION BY col(s) ]
    ORDER BY col(s) [ASC | DESC], ...
  )
  ```

## Concepts

* SQL Ranking window functions assign a ranking, or row number, to each row in a result set based on a specified ordering. 
* The four ranking window functions we discussed in this lesson are `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()`, and `NTILE(n)`. 
* These functions are helpful in analytical queries and are often used with other functions to generate complex ranking and grouping queries.
* The `OVER()` clause defines the window or partition over which the ranking is calculated, and the `PARTITION BY` and `ORDER BY` subclauses are used to group and sort the data within the window.

## References
* [Ranking Functions](https://learn.microsoft.com/en-us/sql/t-sql/functions/ranking-functions-transact-sql?view=sql-server-ver15)
* [Window Functions in PostgreSQL](https://www.postgresql.org/docs/12/functions-window.html)

#DQ

# Offset Window Functions
Learn how to use offset window functions like LEAD(), LAG(), NTH_VALUE(), and more to analyze data and solve real-world analytics problems. Perfect for SQL learners.

* Perform multi-row operations using the LAG() and LEAD() functions
* Identify the first, last, or nth value within a window using offset window functions
* Apply offset window functions to solve real-world problems

##  Introduction to Offset Window Functions
In the previous lessons, we’ve seen how window functions have revolutionized how we query and analyze data. 
Offset window functions provide access to individual rows within a partition and can be divided into two types: one type points to another row based on the current row, while the other type points to a row based on an absolute position, regardless of the current row.  
To understand better the difference between the two types of offset window functions, imagine that you have just arrived at a downtown hotel and are in the mood for a cup of coffee. You decide to go out and look for the closest Starbucks. As you are walking, you come across a guy and ask him for the address of the nearest Starbucks. He could respond in two different ways:

1. By giving directions to a place based on your current location. For example, he might say, "There's a really good Starbucks, just two stores to the north of where we are right now." In this case, the directions are **relative** to your current location. 
2. By giving directions to a place based on a fixed reference point, like a street or landmark. For example, he might say, "There's an awesome Starbucks at the corner of 6th and Spring Street." In this case, the directions are  **independent** of where you're currently located.

Offset window functions support both types of directions, relative and absolute:

* The `LEAD()` and `LAG()` functions come under the **relative** category since they use the positional offset from the current row.
* The `FIRST_VALUE()`, `LAST_VALUE()`, and `NTH_VALUE()` functions belong to the **absolute** category since they use an offset from the frame boundaries, making them **independent** of the current row's position.

This lesson will demonstrate how these functions allow you to efficiently include any columns from other rows in your query results without requiring a self-join.  

Also, we'll discuss how offset window functions can be used to solve real-world problems.  

After you finish this lesson, you'll have a good understanding of the following:

* The definition and categories of offset window functions
* The syntax and usage of offset window functions  
* How to solve real-world problems using offset window functions


Let’s get started.

## The LEAD() Function
The first offset window function we'll discuss is `LEAD()`. The `LEAD()` function returns the values for a row at a specified offset below the current row in the window partition. The default offset is 1 if not specified.
Let’s look at the function syntax before examining it in more detail. 

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/776-909/2.1-m776.svg">
</center>

```sql
LEAD (expression, offset, default_value) 
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

* `expression`: The column or expression on which the `LEAD()` function operates.
* `offset` (optional): A positive integer that defines the number of rows forward from the current row.
* `default_value` (optional): the value to return if the next row does not exist. The default is `NULL`.
* ‍`PARTITION BY‍`: an optional subclause that divides the result set into partitions based on the values of one or more columns or expressions.
* `ORDER BY`: A mandatory subclause that specifies the column or expression used to sort the rows within each partition.

The `LEAD()` function returns the column value in the row that follows the current row by the specified offset. If no such row exists, the function returns the default value, `NULL`.

Notice that while the `PARTITION BY` subclause is optional — enclosed in square brackets — the `ORDER BY` subclause is crucial in determining the sequence of rows in the partition. It specifies the column or expression used to order the rows, making it possible to identify the previous and next rows.

Due to the nature of the `LEAD()` function, which is designed to access specific rows at a fixed offset from the current row, framing cannot be implemented in the window function. The same applies to the `LAG()` function, which we'll discuss on the next screen, as it also focuses on accessing rows at a set distance from the current row.

Consider the first five rows of the following table, `phone_sales_quantity_by_month`, containing the sales data for six months of 2022 for two brands, Samsung and Apple. The number of phone sales for each brand is given in the `quantity` column for each sales date specified in the `sales_date` column.

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>110</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>117</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>75</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>60</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>85</td>
</tr>
</table>
</body>
</html>


Let’s calculate the difference between the sales quantity for each brand between the current and next month.
```sql
SELECT *,
       LEAD(quantity) OVER(PARTITION BY brand
           ORDER BY sales_date) as next_month_sales,
       LEAD(quantity) OVER(PARTITION BY brand
           ORDER BY sales_date) - quantity as sales_diff
  FROM phone_sales_quantity_by_month;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
  <th>next_month_sales</th>
  <th>sales_diff</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>110</td>
  <td>60</td>
  <td>-50</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>60</td>
  <td>85</td>
  <td>25</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>85</td>
  <td>134</td>
  <td>49</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>134</td>
  <td>90</td>
  <td>-44</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>90</td>
  <td>100</td>
  <td>10</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>100</td>
  <td>null</td>
  <td>null</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>117</td>
  <td>75</td>
  <td>-42</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>75</td>
  <td>86</td>
  <td>11</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>86</td>
  <td>124</td>
  <td>38</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>124</td>
  <td>80</td>
  <td>-44</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>80</td>
  <td>89</td>
  <td>9</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>89</td>
  <td>null</td>
  <td>null</td>
</tr>
</table>
</body>
</html>

The `LEAD()` function is applied over the `quantity` column and ordered by `sales_date` within each `brand` partition. The result of `LEAD(quantity)` returns the value of `quantity` in the next row within the partition. By subtracting `quantity` from the result of `LEAD(quantity)`, we can obtain the difference in sales quantity between the current and next month, which is aliased as `sales_diff`.

From a business perspective, analyzing the differences between sales quantities for each brand between consecutive months can provide valuable insights. Positive and increasing differences suggest a growing sales trend, while negative differences can raise concerns about declining performance.

Note that row 6 is the final row of the first partition — the Apple brand.  Since there's no row past row six in the partition and `next_month_sales` and `sales_diff` are `NULL` for row six, there are several `NULL` values in the results.

- - -

**NOTE**

Dataquest code runner displays a blank cell instead of `NULL`.

- - -
### Instructions
Using the `phone_sales_quantity_by_month` table, write a query that returns the sales data for each brand and month and the percentage change in sales quantity between the current and next month.

1. Select the `brand`, `sales_date`, and `quantity` columns from the `phone_sales_quantity_by_month` table.
2. Use the `LEAD()` function, which is applied over the `quantity` column and ordered by `sales_date` within each `brand` partition. The result of `LEAD(quantity)` returns the value of the quantity in the next row within the partition.
   * Alias this result as `next_month_sales`.
3. Create a column called `sales_percentage_change`.
   * Subtract `quantity` from the result of `LEAD(quantity)` to obtain the difference in sales quantity between the current and next month.
   * Divide the result of the above subtraction by the current quantity and multiply it by 100 to get the sales percentage change. Use the cast operator (`::`) to convert the expression data type to `numeric` data type.
5. Sort the result-set based on the `brand` and `sales_date` columns in ascending order.

### Hint
* Use the following expression to get the sales percentage change:

```sql
(LEAD(quantity) OVER (PARTITION BY brand
    				  ORDER BY sales_date)
 - quantity) / quantity::numeric * 100 AS sales_percentage_change
```

```sql
SELECT
  brand,
  sales_date,
  quantity,
  LEAD(quantity) OVER (PARTITION BY brand ORDER BY sales_date) AS next_month_sales,
  (LEAD(quantity) OVER (PARTITION BY brand ORDER BY sales_date) - quantity) / quantity::numeric * 100 AS sales_percentage_change
  FROM
  phone_sales_quantity_by_month
 ORDER BY
  brand,
  sales_date;
```

## The LAG() Function
The second offset window function that we'll explore is `LAG()`. The syntax of the `LAG()` function is the same as the `LEAD()` function; however, unlike the `LEAD()` function, it returns the value of the column from the previous row at a specified offset above the current row in a partition.

Let’s take a look at the basic syntax of the `LAG()` function:

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/776-909/3.1-m776_v2.svg">
</center>

```sql
LAG (expression, offset, default_value) 
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)
```

* `expression`: The column or expression on which the `LAG()` function operates.
* `offset` (optional): A positive integer that defines the number of rows before the current row.
* `default_value` (optional): The value to return if the previous row does not exist. The default is `NULL`.
* ‍`PARTITION BY`: Optionally divides results into groups.
* `ORDER BY`: Required for sorting rows within each group.

The `LAG()` function returns the column value from a preceding row at a specified offset with a default value of `NULL`. While `PARTITION BY` is optional, the `ORDER BY` subclause is necessary to establish the row sequence.

Consider the `phone_sales_revenue_by_month` table containing three columns: `sales_date`, `brand`, and `revenue`. 
The `sales_date` column represents the sales data in the 'YYYY-MM-DD' format. The `brand` column represents the brand of the phone sold, and the `revenue` column represents the revenue generated by each phone brand in that specific month. The table contains data for six months of 2022 for two brands, Apple and Samsung.

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>revenue</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>49950.00</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>36960.00</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>24975.00</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>17970.00</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>28753.00</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>23960.00</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>48921.00</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>43200.00</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>25790.00</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>31240.00</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>70295.00</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>59821.00</td>
</tr>
</table>
</body>
</html>


Let's write a query based on the `phone_sales_revenue_by_month` table to calculate the difference between each month's sales revenue and the sales revenue from the corresponding month in the previous quarter.

The query allows us to compare the June 2022 revenue with the revenue from March 2022. 

```sql
SELECT
    sales_date,
    brand,
    revenue,
    LAG(revenue, 3, 0.0) OVER (PARTITION BY brand ORDER BY sales_date) AS prev_quarter_revenue,
    revenue - LAG(revenue, 3, 0.0) OVER (PARTITION BY brand ORDER BY sales_date) AS difference
  FROM
    phone_sales_revenue_by_month;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>revenue</th>
  <th>prev_quarter_revenue</th>
  <th>difference</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>49950.00</td>
  <td>0</td>
  <td>49950</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>36960.00</td>
  <td>0</td>
  <td>36960</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>24975.00</td>
  <td>0</td>
  <td>24975</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>17970.00</td>
  <td>49950</td>
  <td>-31980</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>28753.00</td>
  <td>36960</td>
  <td>-8207</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>23960.00</td>
  <td>24975</td>
  <td>-1015</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>48921.00</td>
  <td>0</td>
  <td>48921</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>43200.00</td>
  <td>0</td>
  <td>43200</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>25790.00</td>
  <td>0</td>
  <td>25790</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>31240.00</td>
  <td>48921</td>
  <td>-17681</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>70295.00</td>
  <td>43200</td>
  <td>27095</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>59821.00</td>
  <td>25790</td>
  <td>34031</td>
</tr>
</table>
</body>
</html>

As we see in the query above, the `LAG()` function retrieves the previous quarter's revenue using an offset of 3. A default value of 0 is used when no data is available for the corresponding month in the previous quarter.  The lagged value from the current month's revenue is subtracted to get the final result.

 
Undoubtedly, the above query helps the business team understand how revenue changes over time and allows them to make informed decisions. 

### Instructions
Let's assume you're working for a cell phone store that sells two popular smartphones, Apple and Samsung. One day your manager asks you to analyze monthly sales revenue changes for the two brands over six months.
After investigating, you decide to use the `LAG()` offset window function and a `CASE` statement to compare sales revenue between months.

1. Write the query against the `phone_sales_revenue_by_month` table and select the `sales_date`, `brand`, and `revenue` columns.
2. Use a `CASE` statement to classify the revenue change as follows:
   * "Increase" if the current month's revenue is higher than the previous month's revenue.
   * "Decrease" if the current month's revenue is lower than the previous month's revenue.
   * "No Change" if the revenue is the same as the previous month or there is no previous month.
   * The labels should be displayed in a column called `revenue_change`.
3. Use the `LAG()` function inside the `CASE` statement to calculate the current and previous months' revenue differences.
   * Partition the data by brand.
   * Order the partitions by `sales_date`.

### Hint
* To retrieve the difference between the current month's revenue and the previous month's revenue, use the following expression:

```sql
revenue - LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date)
```

* Label the revenue change as "Increase" if the current month's revenue is higher than the previous month's revenue, "Decrease" if the current month's revenue is lower than the previous month's revenue, and "No Change" if the revenue is the same as the previous month. Use the following syntax for the `CASE` statement:

```sql
CASE
        WHEN revenue - LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date) > 0 THEN 'Increase'
        WHEN revenue - LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date) < 0 THEN 'Decrease'
        ELSE 'No Change'
    END AS revenue_change
```

```sql
SELECT
    sales_date,
    brand,
    revenue,
    CASE
        WHEN revenue - LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date) > 0 THEN 'Increase'
        WHEN revenue - LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date) < 0 THEN 'Decrease'
        ELSE 'No Change'
    END AS revenue_change
FROM
    phone_sales_revenue_by_month;
```

## The FIRST_VALUE() Function
In the previous section, we covered the `LEAD()` and `LAG()` offset window functions, which allow us to include any column from a given number of rows away from the current row. Since these functions are relative to the current row, they're also called row offset window functions.

In contrast, the `FIRST_VALUE()`, `LAST_VALUE()`, and `NTH_VALUE()` functions are not dependent on the current row and allow us to include any column from the first, last, or nth row of the partition. These functions are also called frame offset window functions because they specify the offset with respect to the beginning or end of the window, and framing becomes relevant.

Let's get started with the `FIRST_VALUE()` function.

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/776-909/4.1-m776_v2.svg">
</center>

The `FIRST_VALUE()` function retrieves the value of the first row within a partition.
Here's the basic syntax of the function:
```sql
FIRST_VALUE (expression)
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
    [framing_clause]
)
```

Given that we've already discussed the general syntax of the offset window functions, let's shift our focus to the last part of the syntax above, `[framing_clause]`.

The `LEAD()` and `LAG()` functions support window partitioning and ordering, but not window framing. This is reasonable since the offset is relative to the current row. When using the `FIRST_VALUE()` and `LAST_VALUE()` functions, window framing comes into play because the offset is relative to the start and end of the window.

We can rely on the default framing when using the `FIRST_VALUE()` function. As we discussed in the *Window Function Framing lesson* when we don't explicitly indicate a window frame clause, the default framing is as follows:

`RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`

This means that the `FIRST_VALUE()` function returns the first row of the partition with the default window framing.

While relying on the default framing makes sense, and the `FIRST_VALUE()` function returns a correct result, using `ROWS` instead of `RANGE` is more efficient for the database server's performance.

Now we'll explore how the `FIRST_VALUE` function works in practice. 

Consider the `employees` table containing details of nine employees working for different departments in a company:

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>employee_id</th>
  <th>last_name</th>
  <th>first_name</th>
  <th>department</th>
  <th>title</th>
  <th>hire_date</th>
  <th>salary</th>
</tr>
<tr>
  <td>1</td>
  <td>Adams</td>
  <td>Andrew</td>
  <td>Management</td>
  <td>General Manager</td>
  <td>2002-08-13</td>
  <td>108000.00</td>
</tr>
<tr>
  <td>2</td>
  <td>Edwards</td>
  <td>Nancy</td>
  <td>Sales</td>
  <td>Sales Manager</td>
  <td>2002-04-30</td>
  <td>98900.00</td>
</tr>
<tr>
  <td>3</td>
  <td>Peacock</td>
  <td>Jane</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2002-03-31</td>
  <td>87000.00</td>
</tr>
<tr>
  <td>4</td>
  <td>Park</td>
  <td>Margaret</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-05-02</td>
  <td>69800.00</td>
</tr>
<tr>
  <td>5</td>
  <td>Johnson</td>
  <td>Steve</td>
  <td>Sales</td>
  <td>Sales Support Agent</td>
  <td>2003-10-16</td>
  <td>76500.00</td>
</tr>
<tr>
  <td>6</td>
  <td>Mitchell</td>
  <td>Michael</td>
  <td>IT</td>
  <td>IT Manager</td>
  <td>2003-10-16</td>
  <td>89900.00</td>
</tr>
<tr>
  <td>7</td>
  <td>King</td>
  <td>Robert</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-01-01</td>
  <td>67800.00</td>
</tr>
<tr>
  <td>8</td>
  <td>Callahan</td>
  <td>Laura</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-03-03</td>
  <td>78000.00</td>
</tr>
<tr>
  <td>9</td>
  <td>Edward</td>
  <td>John</td>
  <td>IT</td>
  <td>IT Staff</td>
  <td>2004-09-18</td>
  <td>75900.00</td>
</tr>
</table>
</body>
</html>

Let's assume that Adams, the company's manager, has requested a report to identify the first hired employees in the sales and IT departments. This report will be used to give gifts to those employees. The request can be fulfilled using the following query:

```sql
SELECT department, first_name ||' ' || last_name as full_name, hire_date,
    FIRST_VALUE(first_name ||' ' || last_name) OVER (
      PARTITION BY department
      ORDER BY hire_date
    ) AS hired_first
  FROM employees
 WHERE department IN ('Sales', 'IT');
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>department</th>
  <th>full_name</th>
  <th>hire_date</th>
  <th>hired_first</th>
</tr>
<tr>
  <td>IT</td>
  <td>Michael Mitchell</td>
  <td>2003-10-16</td>
  <td>Michael Mitchell</td>
</tr>
<tr>
  <td>IT</td>
  <td>Robert King</td>
  <td>2004-01-01</td>
  <td>Michael Mitchell</td>
</tr>
<tr>
  <td>IT</td>
  <td>Laura Callahan</td>
  <td>2004-03-03</td>
  <td>Michael Mitchell</td>
</tr>
<tr>
  <td>IT</td>
  <td>John Edward</td>
  <td>2004-09-18</td>
  <td>Michael Mitchell</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Jane Peacock</td>
  <td>2002-03-31</td>
  <td>Jane Peacock</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Nancy Edwards</td>
  <td>2002-04-30</td>
  <td>Jane Peacock</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Margaret Park</td>
  <td>2003-05-02</td>
  <td>Jane Peacock</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Steve Johnson</td>
  <td>2003-10-16</td>
  <td>Jane Peacock</td>
</tr>
</table>
</body>
</html>


The above result shows that Michael Mitchell and Jane Peacock are eligible for the gifts, but this may be different from the result format you were looking for. The manager will probably be delighted if you only declare the details of the two individuals that should receive the gifts. 

In the upcoming exercise, we'll implement what was just discussed. 




### Instructions
Write a more complicated query, including a window frame specification to return what Adams expects.

1. Create a CTE and use `FIRST_VALUE()` with a window frame specification to determine the first hire for each department ('Sales', 'IT').
   * Partition the data based on the `department` column and sort the data within each partition by the `hire_date` column in the `OVER()` clause.
2. Filter the results to only include employees with the first hire date for their department.
3. Return only `department`, `full_name`, and `hire_date` columns.

### Hint
* Create the field `full_name` by concatenating `first_name` and `last_name` together using the `||` operator.
* Make sure to use a WHERE filter on the Sales and IT departments.

```sql
WITH hired_first AS (
    SELECT department, first_name ||' '|| last_name as full_name, hire_date,
        FIRST_VALUE(hire_date) OVER (
            PARTITION BY department
            ORDER BY hire_date) AS first_hire_date
    FROM employees
    WHERE department IN ('Sales', 'IT')
)
SELECT department, full_name, hire_date
  FROM hired_first
 WHERE hire_date = first_hire_date;
```

## The LAST_VALUE() Function
The `LAST_VALUE()` function retrieves the value of the last row within a partition.
Here's the basic syntax of the function:

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/776-909/5.1-m776_v2.svg">
</center>

```sql
LAST_VALUE (expression)
OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
    [framing_clause]
)
```

We can observe that the syntax for the `FIRST_VALUE()` and `LAST_VALUE()` functions are identical except in function names.

However, the bad news is that with `LAST_VALUE()`, we can't rely on the default window framing since the last row of the default frame is the current row, which gives an incorrect result. 

Let's try using the `LAST_VALUE()` function with partitioning and ordering without an explicit window frame to get the last hire date for each department for employees in the 'Sales' and 'IT' departments.

```sql
SELECT department, first_name ||' '|| last_name as full_name, hire_date,
        LAST_VALUE(hire_date) OVER (
            PARTITION BY department
            ORDER BY hire_date) AS last_hire_date
  FROM employees
 WHERE department IN ('Sales', 'IT');
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>department</th>
  <th>full_name</th>
  <th>hire_date</th>
  <th>last_hire_date</th>
</tr>
<tr>
  <td>IT</td>
  <td>Michael Mitchell</td>
  <td>2003-10-16</td>
  <td>2003-10-16</td>
</tr>
<tr>
  <td>IT</td>
  <td>Robert King</td>
  <td>2004-01-01</td>
  <td>2004-01-01</td>
</tr>
<tr>
  <td>IT</td>
  <td>Laura Callahan</td>
  <td>2004-03-03</td>
  <td>2004-03-03</td>
</tr>
<tr>
  <td>IT</td>
  <td>John Edward</td>
  <td>2004-09-18</td>
  <td>2004-09-18</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Jane Peacock</td>
  <td>2002-03-31</td>
  <td>2002-03-31</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Nancy Edwards</td>
  <td>2002-04-30</td>
  <td>2002-04-30</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Margaret Park</td>
  <td>2003-05-02</td>
  <td>2003-05-02</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Steve Johnson</td>
  <td>2003-10-16</td>
  <td>2003-10-16</td>
</tr>
</table>
</body>
</html>

While we were expecting the query above to return every employee's department, full name, and hire date, alongside the last hire date in each department, it returns the current row's hire date in the `last_hire_date` column. 
To fix the issue, we have to use an explicit window frame specification with `UNBOUNDED FOLLOWING` as the upper boundary of the frame, as follows:

```sql
SELECT department, first_name ||' '|| last_name as full_name, hire_date,
        LAST_VALUE(hire_date) OVER (
            PARTITION BY department
            ORDER BY hire_date
            ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS first_hire_date
  FROM employees
 WHERE department IN ('Sales', 'IT');
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>department</th>
  <th>full_name</th>
  <th>hire_date</th>
  <th>first_hire_date</th>
</tr>
<tr>
  <td>IT</td>
  <td>Michael Mitchell</td>
  <td>2003-10-16</td>
  <td>2004-09-18</td>
</tr>
<tr>
  <td>IT</td>
  <td>Robert King</td>
  <td>2004-01-01</td>
  <td>2004-09-18</td>
</tr>
<tr>
  <td>IT</td>
  <td>Laura Callahan</td>
  <td>2004-03-03</td>
  <td>2004-09-18</td>
</tr>
<tr>
  <td>IT</td>
  <td>John Edward</td>
  <td>2004-09-18</td>
  <td>2004-09-18</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Jane Peacock</td>
  <td>2002-03-31</td>
  <td>2003-10-16</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Nancy Edwards</td>
  <td>2002-04-30</td>
  <td>2003-10-16</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Margaret Park</td>
  <td>2003-05-02</td>
  <td>2003-10-16</td>
</tr>
<tr>
  <td>Sales</td>
  <td>Steve Johnson</td>
  <td>2003-10-16</td>
  <td>2003-10-16</td>
</tr>
</table>
</body>
</html>

### Instructions
Write a query against the `phone_sales_revenue_by_month` table that calculates the percentage change in revenue from each brand's first and last month of sales.

This query is among the most complex queries you've written so far. Remember, trial and error is an essential aspect of the coding process, so it's perfectly acceptable if it takes multiple attempts to succeed!

Follow the step-by-step guide to implement the query.

1. Select the `sales_date`, `brand`, and `revenue` columns from the `phone_sales_revenue_by_month` table.
2. Calculate the percentage change in revenue from each brand's first month of sales using the `FIRST_VALUE()` function.
   * Partition the data by `brand`.
   * Order the data by `sales_date` in ascending order.
   * Use the `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` to include all rows from the beginning of the partition  to the current row.
   * Calculate the percentage change as the difference between revenue and the first value of revenue in the partition, divided by the first value of revenue in the partition, multiplied by 100. Round the result to two decimal places.
   * Assign the result to a new column called `first_month_pct_change`.
3. Calculate the percentage change in revenue from the last month of sales for each brand using the `LAST_VALUE()` function.
   * Partition the data by `brand`.
   * Order the data by `sales_date` in ascending order.
   * Define a frame including all rows from the current row to the end of the partition.
   * Calculate the percentage change as the difference between revenue and the last value of revenue in the partition, divided by the last value of revenue in the partition, multiplied by 100. Round the result to two decimal places.
   * Assign the result to a new column called `last_month_pct_change`.

### Hint
* Use the following frame specification to include all rows from the beginning of the partition up to the current row.

`ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT`

* Use the following frame specification to include all rows from the current row up to the end of the partition:

`ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`

* Remember that the `ROUND()` function will need to wrap around the entire expression to make the output easier to read and interpret.

```sql
SELECT
    sales_date,
    brand,
    revenue,
    ROUND((revenue - FIRST_VALUE(revenue) OVER (
        PARTITION BY brand
        ORDER BY sales_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )) / FIRST_VALUE(revenue) OVER (
        PARTITION BY brand
        ORDER BY sales_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) * 100,2) AS first_month_pct_change,
    ROUND((revenue - LAST_VALUE(revenue) OVER (
        PARTITION BY brand
        ORDER BY sales_date
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    )) / LAST_VALUE(revenue) OVER (
        PARTITION BY brand
        ORDER BY sales_date
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    ) * 100,2) AS last_month_pct_change
FROM
    phone_sales_revenue_by_month;
```

## The NTH_VALUE() Function
The last offset window function we'll discuss is `NTH_VALUE()`. 

The `NTH_VALUE()` function retrieves a value of a column or expression in a specified offset from the first row in the window frame.

<center>
<img src="https://s3-us-east-2.amazonaws.com/dq-authoring-tmp-data/776-909/6.1-m776.svg">
</center>

The basic syntax of the `NTH_VALUE()` function is as follows:

```sql
NTH_VALUE (expression, offset) OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC|DESC], ... 
    [framing_clause]
)
```
* `NTH_VALUE()` takes two mandatory arguments:

* 'expression': The column or expression on which the function operates.
* 'offset': The position of the value to be retrieved, starting from 1.


* `PARTITION BY partition_expression` is optional and divides the dataset into partitions to which the NTH_VALUE() function is applied.

* `ORDER BY sort_expression` is mandatory and sorts the rows within each partition. If ORDER BY is omitted, the default frame consists of all rows in the partition.

* `framing_clause` is required if an ORDER BY clause is used. 

To better understand how this function works, let's assume a company has been experiencing difficulties with employee retention and decided to revise the company salary structure to address the problem. They've decided to analyze the performance of employees who receive the third highest salary in each department. They believe the analysis result allows them to adjust and improve employee satisfaction and retention. 

Our task is to write a query that retrieves the details of employees who receive the third highest salary.

We'll use a CTE to identify the third highest salary for each department and display it alongside the employee details, then filter the results to only include employees from the IT and Sales departments.

```sql
WITH third_highest_salary AS
    (
        SELECT department, first_name ||' '|| last_name as full_name, hire_date, salary, 
        NTH_VALUE(salary, 3) OVER (
    PARTITION BY department
    ORDER BY salary DESC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
) AS third_highest_salary
FROM employees
WHERE department in ('IT','Sales')
ORDER BY department, hire_date
    )

SELECT full_name, department, hire_date, salary
FROM third_highest_salary
WHERE salary = third_highest_salary;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>full_name</th>
  <th>department</th>
  <th>hire_date</th>
  <th>salary</th>
</tr>
<tr>
  <td>John Edward</td>
  <td>IT</td>
  <td>2004-09-18</td>
  <td>75900.00</td>
</tr>
<tr>
  <td>Steve Johnson</td>
  <td>Sales</td>
  <td>2003-10-16</td>
  <td>76500.00</td>
</tr>
</table>
</body>
</html>

The following expression is the core part of the above query:

```sql
NTH_VALUE(salary, 3) OVER (
    PARTITION BY department
    ORDER BY salary DESC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS third_highest_salary
```

The `NTH_VALUE()` function returns the third highest salary by ordering the salaries in descending order within each department and returning the value of the third row. The `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` clause defines the window frame as including all rows in each department, which allows the `NTH_VALUE()` function to calculate the third highest salary over all rows in each department.


### Instructions
Suppose the cell phone store wants to analyze the performance of the **second-best** performing month for each brand. 

1. Write a query against the `phone_sales_revenue_by_month` table that retrieves the following details for the second-best performing month for each brand in 2022:
   * `brand`
   * `sales_date`
   * `second_highest_revenue`
2. Create a common table expression (CTE).
   * Use the `NTH_VALUE()` in the `SELECT` statement to calculate the second-highest revenue for each brand.
     * Partition the data by `brand` and sort them by `revenue` in descending order. 
     * Define a frame that includes all rows in the partition.
     * Alias the result in a column called `second_highest_revenue`.
3. Reference the CTE in the main query and filter the result set only to include the rows where the revenue equals the second-highest revenue calculated in the CTE. 

### Hint
* Retrieve the second-highest revenue month for each brand in the CTE using the `NTH_VALUE()` window function as follows:

`NTH_VALUE(revenue, 2)`
* Partition the window by the `brand` column and order it by the `revenue` column in descending order.
* Include all rows within the partition for the calculation using the following frame specification:

`ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`.
* Use the following statement in the main query to filter the result set to only returns the rows where the revenue equals the second-highest revenue.

```sql
WHERE revenue = second_highest_revenue
```

```sql
WITH second_highest_rev AS
(
    SELECT brand, sales_date, revenue,
           NTH_VALUE(revenue, 2) OVER (
               PARTITION BY brand
               ORDER BY revenue DESC
               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
           ) AS second_highest_revenue
    FROM phone_sales_revenue_by_month
)
SELECT brand, sales_date, second_highest_revenue
  FROM second_highest_rev
 WHERE revenue = second_highest_revenue;
```

## Offset Window Functions in Solving Real-World Problems
The SQL offset window functions provide powerful tools for solving real-world problems that require sequential data analysis.


### First Scenario - Anomaly Detection
The `LAG()` and `LEAD()` functions can detect anomalies in time-series financial data. Let's try them in practice. 


Let's use the `LAG()` and `LEAD()` functions to detect anomalies in the cell phone sales data stored in the `phone_sales_revenue_by_month` table.

We want to detect anomalies in monthly sales data for each brand by comparing each month's revenue with the revenue for the previous and next months.

Anomalies in sales revenue could point to months with significant revenue growth or decline. Businesses can analyze the factors driving these changes and adjust their marketing strategies accordingly. 

To do this, we can use the `LAG()` and `LEAD()` functions to retrieve the sales data for the previous and next months, respectively, and then compare them with the sales data for the current month.
Here's the query that uses the `LAG()` and `LEAD()` functions to detect anomalies in the monthly sales data:

```sql
SELECT
  sales_date,
  brand,
  revenue,
  CASE
    WHEN revenue > LAG(revenue) OVER (PARTITION BY brand ORDER BY sales_date) * 1.4
        OR revenue > LEAD(revenue) OVER (PARTITION BY brand ORDER BY sales_date) * 1.4
    THEN 'Anomaly'
    ELSE 'Normal'
  END AS sales_status
FROM phone_sales_revenue_by_month
ORDER BY brand, sales_date;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>revenue</th>
  <th>sales_status</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>49950.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>36960.00</td>
  <td>Anomaly</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>24975.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>17970.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>28753.00</td>
  <td>Anomaly</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>23960.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>48921.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>43200.00</td>
  <td>Anomaly</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>25790.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>31240.00</td>
  <td>Normal</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>70295.00</td>
  <td>Anomaly</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>59821.00</td>
  <td>Normal</td>
</tr>
</table>
</body>
</html>


The query above uses a `CASE` statement to compare the current month's revenue with the previous and next months' revenues; if the current month's revenue is forty percent higher than the previous and next months' revenues, the current month's revenue is classified as `Anomaly`; otherwise, it's classified as `Normal`.


Combining common table expressions with the offset window functions allows us to write complicated analytical queries. 

Challenge your knowledge with a hands-on exercise.


### Instructions
The cell phone store management has defined revenue ranges as follows:

* Golden Month: revenue >= 60K USD
* Silver Month: 49K USD<= revenue< 60K USD
* Bronze Month: 20K USD<= revenue< 49K USD

Let's assume the store manager is keen to know the first month each brand's revenue falls into the **silver** month range.

Write a query against the `phone_sales_revenue_by_month` table that addresses the request.

1. Create a Common Table Expression (CTE) to filter the table to include only rows with revenue between 49,000 USD and 60,000 USD.
   * Use the `FIRST_VALUE()` function in the CTE to determine the first month each brand's sales revenue falls within the silver range.
2. In the main query, select the `sales_date`, `brand`, and `revenue` columns from the CTE.
3. Add a condition in the WHERE clause to filter the results to show only rows where the revenue is equal to the value calculated in the CTE.

### Hint
* Use the `PARTITION BY` and `ORDER BY` subclauses within the window function to partition the data by `brand` and order it by `sales_date`.
* Use the following statement to filter the data within the CTE according to the silver revenue range.


```sql
WITH sales_revenue AS (
    SELECT *,
           FIRST_VALUE(revenue) OVER (
               PARTITION BY brand ORDER BY sales_date) AS first_month_49K_60K
               FROM phone_sales_revenue_by_month
               WHERE revenue >= 49000 AND revenue < 60000
    )
SELECT sales_date, brand, revenue
  FROM sales_revenue
 WHERE revenue = first_month_49K_60K;
```

## Review
Congratulations on finishing this lesson!

We've learned about the following:


* Different types of offset window functions that operate on a position that is either relative to the current row or relative to the starting or ending boundary of the window frame.
* The syntax and basic usage of the `LEAD()`, `LAG()`, `FIRST_VALUE()`, `LAST_VALUE()`, and `NTH_VALUE()` functions.
* The applications of the functions for performing data analysis and solving real-world problems. 

In the next lesson, we'll learn about the distribution window functions.

## Takeaways
## Syntax
* **LEAD()** returns the value of a column or expression from a row that is offset rows after the current row in the window frame.

  ```
  LEAD(expression, offset [, default]) OVER (
      PARTITION BY partition_expression,... 
      ORDER BY sort_expression [ASC|DESC], ...
  )
  ```

* **LAG()** returns the value of a column or expression from a row that is offset rows before the current row in the window frame.

  ```
  LAG(expression, offset [, default]) OVER (
      PARTITION BY partition_expression,... 
      ORDER BY sort_expression [ASC|DESC], ...
  )
  ```

* **FIRST_VALUE()** returns the value of a column or expression from the first row in the window frame.

  ```
  FIRST_VALUE(expression) OVER (
      PARTITION BY partition_expression,... 
      ORDER BY sort_expression [ASC|DESC], ... 
      [framing_clause])
  ```

* **LAST_VALUE()** returns the value of a column or expression from the last row in the window frame.

  ```
  LAST_VALUE(expression) OVER (
      PARTITION BY partition_expression,...
      ORDER BY sort_expression [ASC|DESC],... 
      [framing_clause])
  ```

* **NTH_VALUE()** returns the value of a column or expression in a given offset in terms of the number of rows from the first or last row in the window frame.

  ```
  NTH_VALUE(expression, offset) OVER (
      PARTITION BY partition_expression,... 
      ORDER BY sort_expression [ASC|DESC], ... 
      [framing_clause])
  ```


## Concepts

* Offset window functions retrieve values from rows before or after the current row or from the first or last row in the window frame. The syntax of these functions is similar, but the specific parameters may differ depending on the function.
* As the `ORDER BY` subclause determines the order in which the rows are processed in the window, it is crucial for the offset window functions to work correctly.
* Partitioning the data into groups with the `PARTITION BY` subclause is optional; however, it becomes essential when using offset window functions to perform calculations within the groups. It allows us to apply the same function to each group separately.


## Resources

* [Window Functions in PostgreSQL](postgresql.org/docs/current/functions-window.html)

* [T-SQL Window Functions](https://www.microsoftpressstore.com/store/t-sql-fundamentals-9780138102104)

#DQ

# Distribution Window Functions
This lesson covers two types of distribution window functions: rank distribution functions and inverse distribution functions. You’ll have a good understanding of how to use them for statistical analysis, how to calculate the value at a specified percentile, and how to solve real-world problems with distribution window functions.

* Distinguish between CUME_DIST(), PERCENT_RANK(), PERCENTILE_DISC(), and PERCENTILE CONT()
* Apply rank distribution window functions for statistical analysis
* Utilize inverse distribution functions to determine the value at a particular percentile
* Implement the WINDOW clause to create concise and manageable queries

## Distribution Window Functions
The last lesson of the SQL window functions course delves into distribution or statistical window functions, which provide insights into data distribution and are used chiefly for statistical analysis.

The distribution window functions have diverse applications across various domains, including marketing, finance, healthcare, sports, and education. They allow data analysts to gain valuable insights from data and make data-driven decisions.
For example, these functions can help marketers identify the top-performing products, campaigns, or customers based on their sales or engagement metrics, allowing them to optimize their marketing strategy. Also, they can be used for calculating the percentiles of medical data such as blood pressure, BMI, or cholesterol levels, helping physicians or researchers diagnose diseases or assess the health status of patients.

We'll cover two types of distribution window functions:

1. Rank distribution functions are used to calculate the relative rank of a specific row in a window partition based on the ordering specified in the `OVER()` clause.

    
* `PERCENT_RANK()`


* `CUME_DIST()`

        
2. Inverse distribution functions are used to calculate the value at a specified percentile in a group based on the ordering specified in the `WITHIN GROUP()` clause.

    
* `PERCENTILE_CONT()`


* `PERCENTILE_DISC()`

        

After this lesson, you'll have a good understanding of the following:

* How to use rank distribution window functions in SQL for statistical analysis.


* How to use inverse distribution functions to calculate the value at a specified percentile.


* How to solve real-world problems with distribution window functions


* Using the `WINDOW` clause to write more manageable queries.

    

Let's get started.


## The Rank Distribution Functions - CUME_DIST()
As mentioned, two variants of rank distribution functions exist: `CUME_DIST()` and `PERCENT_RANK()`. They compute a row's relative rank in the window partition and return a ratio between 0 and 1.

The first rank distribution function that we'll discuss is `CUME_DIST()`.

### The `CUME_DIST()` Function

The `CUME_DIST()` function, which takes no argument, calculates the cumulative distribution of values in a column specified in the `ORDER BY` subclause. In other words, assuming the data is sorted in ascending order, the cumulative distribution of a value in a particular row is determined as the number of rows with values less than or equal to that value divided by the number of rows existing in the window partition.

The basic syntax of the `CUME_DIST()` function is as follows:

```sql

CUME_DIST() OVER ( [PARTITION BY expression, ... ]
                   ORDER BY expression [ASC | DESC], ... )
```

* ‍`PARTITION BY‍`: an optional subclause that divides the result set into partitions based on the values of one or more columns or expressions.
* `ORDER BY`: a mandatory subclause that specifies the column or expression used to sort the rows within each partition.


Let's see how the function works.

Consider the `trips` table containing twelve bike trips as follows:

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>end_date</th>
  <th>duration</th>
  <th>start_station_number</th>
  <th>start_station</th>
  <th>end_station_number</th>
  <th>end_station</th>
  <th>bike_number</th>
  <th>member_type</th>
  <th>rider_rating</th>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>2017-10-01 03:29:00.000000</td>
  <td>1253</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31041</td>
  <td>Prince St &amp; Union St</td>
  <td>W23272</td>
  <td>Member</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>2017-10-01 05:26:00.000000</td>
  <td>1476</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31010</td>
  <td>S Glebe &amp; Potomac Ave</td>
  <td>W23254</td>
  <td>Member</td>
  <td>2</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>2017-10-01 05:12:00.000000</td>
  <td>650</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31011</td>
  <td>23rd &amp; Crystal Dr</td>
  <td>W00143</td>
  <td>Member</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>2017-10-02 03:47:00.000000</td>
  <td>987</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31249</td>
  <td>Jefferson Memorial</td>
  <td>W21096</td>
  <td>Member</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>2017-10-03 09:47:00.000000</td>
  <td>797</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31518</td>
  <td>New York Ave &amp; Hecht Ave NE</td>
  <td>W20095</td>
  <td>Member</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>2017-10-03 12:23:00.000000</td>
  <td>1390</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31247</td>
  <td>Jefferson Dr &amp; 14th St SW</td>
  <td>W22965</td>
  <td>Casual</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>2017-10-04 05:11:00.000000</td>
  <td>797</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31503</td>
  <td>Florida Ave &amp; R St NW</td>
  <td>W23052</td>
  <td>Casual</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>2017-10-04 05:36:00.000000</td>
  <td>918</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31506</td>
  <td>1st &amp; Rhode Island Ave NW</td>
  <td>W22051</td>
  <td>Casual</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>2017-10-04 06:22:00.000000</td>
  <td>918</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31126</td>
  <td>11th &amp; Girard St NW</td>
  <td>W23268</td>
  <td>Member</td>
  <td>4</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>2017-10-04 08:45:00.000000</td>
  <td>918</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31235</td>
  <td>19th St &amp; Constitution Ave NW</td>
  <td>W22517</td>
  <td>Casual</td>
  <td>5</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>2017-10-05 08:11:00.000000</td>
  <td>202</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31009</td>
  <td>27th &amp; Crystal Dr</td>
  <td>W20184</td>
  <td>Member</td>
  <td>3</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>2017-10-05 08:33:00.000000</td>
  <td>1482</td>
  <td>31002</td>
  <td>20th &amp; Crystal Dr</td>
  <td>31633</td>
  <td>Independence Ave &amp; L&#39;Enfant Plaza SW/DOE</td>
  <td>W00895</td>
  <td>Casual</td>
  <td>4</td>
</tr>
</table>
</body>
</html>


Now, let's try the following query to see the cumulative distribution of the `duration` column:

```sql
SELECT start_date, bike_number, duration,
       ROUND(CUME_DIST() OVER(ORDER BY duration)::numeric, 2) AS cume_dist
  FROM trips;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>duration</th>
  <th>cume_dist</th>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>202</td>
  <td>0.08</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>650</td>
  <td>0.17</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>797</td>
  <td>0.33</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>797</td>
  <td>0.33</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>918</td>
  <td>0.58</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>918</td>
  <td>0.58</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>918</td>
  <td>0.58</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>987</td>
  <td>0.67</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>1253</td>
  <td>0.75</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>1390</td>
  <td>0.83</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>1476</td>
  <td>0.92</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>1482</td>
  <td>1</td>
</tr>
</table>
</body>
</html>

- - -

**NOTE**

The `ROUND()` function rounds the cumulative distribution value to two decimal places.

It's worth noting that without casting the result to a numeric data type, the `ROUND()` function would not apply to the output of the `CUME_DIST()` function.
- - -

We may ask why the value returned by `CUME_DIST()` for the first row is not zero. This is because the first value is always included in the distribution. Hence the first value of the `CUME_DIST()` function represents the proportion of rows in the window partition whose values are less than or equal to the first value. 

The above query calculates the cumulative distribution of each duration value with respect to the other values in the `duration` column within the window sorted ascendingly based on the `duration` column.

The first row shows that the duration value of 202 has a cumulative distribution of 0.08. This means that approximately 8.0% of the duration values in the table are less than or equal to 202. Similarly, the cumulative distribution of the second row's duration is 0.17, which means approximately 17% of the duration values are less than or equal to 650.

The analysis becomes more interesting when we come to the third and fourth rows with duration values of 797. All have the same cumulative distribution of 0.33. This means that approximately 33% of the duration values in the table are less than or equal to 797.

### Instructions
The bike-sharing company wants to promote safe riding habits by encouraging users to prioritize safety while still enjoying bike riding. To do so, they plan to identify the fastest riders for each day and send them a notification to remind them that safety comes first.

For this exercise, we assume that for each day, if the cumulative distribution of a trip duration falls in the top 5% (i.e., has a cumulative distribution greater than or equal to 0.95), the trip is a fast trip, and the trip's rider is considered a fast rider.

Write a query against the `tbl_bikeshare` table to select the fast riders for each day.

1. Create a CTE called "fast_rides" that performs the following steps:
   * Select the `start_date`, `duration`, and `bike_number` columns.
   * Use the `EXTRACT` function to pull the day from the `start_date` column.
   * Calculate the cumulative distribution of trip durations for each day using the `CUME_DIST()` window function, partitioned by day and ordered by duration in descending order.
   * Use a `CASE` statement to assign the label 'Fast Rider' to the riders whose trips' durations fall in the top 5% (i.e., have a cumulative distribution greater than or equal to 0.95). Alias this new column as `fast_riders`.
2. In the main query, select all columns from the "fast_rides" CTE.
3. Filter the main query results to show only the rows labeled 'Fast Rider' in the `fast_riders` column.
4. Order the final result set by `start_date` and `duration`.

### Hint
* Assign the `Fast Rider` label to the fast rider using the following statement:


```sql
CASE
        WHEN CUME_DIST() OVER (PARTITION BY EXTRACT(day from start_date) ORDER BY duration DESC) >= 0.95 THEN 'Fast Rider'
        ELSE ''
 END AS fast_riders
```

```sql
WITH fast_rides AS
    (SELECT
    start_date,
    duration,
    bike_number,
    CUME_DIST() OVER (PARTITION BY EXTRACT(day FROM start_date) ORDER BY duration DESC),
    CASE
        WHEN CUME_DIST() OVER (PARTITION BY EXTRACT(day FROM start_date) ORDER BY duration DESC) >= 0.95 THEN 'Fast Rider'
        ELSE ''
   END AS fast_riders
  FROM tbl_bikeshare
 ORDER BY start_date, duration)
SELECT * FROM fast_rides WHERE fast_riders= 'Fast Rider'
 ORDER BY start_date, duration;
```

## The Rank Distribution Functions - PERCENT_RANK()
The second rank distribution function that we will discuss is `PERCENT_RANK()`.

### The `PERCENT_RANK()` Function

Although the `PERCENT_RANK()` function is similar to the `CUME_DIST()` function, it generates quite different results than the `CUME_DIST()` function.

Let's first look at the basic syntax of the function and then discuss it in detail.

```sql
PERCENT_RANK() OVER ( [PARTITION BY expression, ... ]
                   ORDER BY expression [ASC | DESC], ... )
```

* ‍`PARTITION BY‍`: an optional subclause that divides the result set into partitions based on the values of one or more columns or expressions.
* `ORDER BY`: a mandatory subclause that specifies the column or expression used to sort the rows within each partition.

The `PERCENT_RANK()` function is defined as the rank of a value minus one divided by the number of rows in the window partition minus one. This enables performing operations such as dividing data into quartiles for analytical purposes.

$\text{Percent Rank} = \frac{Rank- 1}{\text{Number of Rows -1}}$

Now, let's try the following query against the `trips` table:

```sql
SELECT start_date, bike_number, duration,
       ROUND(PERCENT_RANK() OVER(ORDER BY duration ASC)::NUMERIC,2) AS percent_rank
  FROM trips;
```
<!DOCTYPE html>
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>start_date</th>
  <th>bike_number</th>
  <th>duration</th>
  <th>percent_rank</th>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W20184</td>
  <td>202</td>
  <td>0</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W00143</td>
  <td>650</td>
  <td>0.09</td>
</tr>
<tr>
  <td>2017-10-04 04:58:00.000000</td>
  <td>W23052</td>
  <td>797</td>
  <td>0.18</td>
</tr>
<tr>
  <td>2017-10-03 09:34:00.000000</td>
  <td>W20095</td>
  <td>797</td>
  <td>0.18</td>
</tr>
<tr>
  <td>2017-10-04 05:21:00.000000</td>
  <td>W22051</td>
  <td>918</td>
  <td>0.36</td>
</tr>
<tr>
  <td>2017-10-04 06:07:00.000000</td>
  <td>W23268</td>
  <td>918</td>
  <td>0.36</td>
</tr>
<tr>
  <td>2017-10-04 08:30:00.000000</td>
  <td>W22517</td>
  <td>918</td>
  <td>0.36</td>
</tr>
<tr>
  <td>2017-10-02 03:30:00.000000</td>
  <td>W21096</td>
  <td>987</td>
  <td>0.64</td>
</tr>
<tr>
  <td>2017-10-01 03:08:00.000000</td>
  <td>W23272</td>
  <td>1253</td>
  <td>0.73</td>
</tr>
<tr>
  <td>2017-10-03 12:00:00.000000</td>
  <td>W22965</td>
  <td>1390</td>
  <td>0.82</td>
</tr>
<tr>
  <td>2017-10-01 05:01:00.000000</td>
  <td>W23254</td>
  <td>1476</td>
  <td>0.91</td>
</tr>
<tr>
  <td>2017-10-05 08:08:00.000000</td>
  <td>W00895</td>
  <td>1482</td>
  <td>1</td>
</tr>
</table>
</body>
</html>


- - -

**NOTE**

Unlike the `CUME_DIST()`, the first value for `PERCENT_RANK()` will always be `0` because the first row is not ranked above any other. In other words, assuming ascending ordering, the `PERCENT_RANK()` function assigns a percentile rank of `0` to the lowest value since it's the only row in the result set with a value less than the others.

It's also worth noting that while `CUME_DIST()` calculates the proportion of values that are less than or equal to a given value, representing the cumulative distribution of the data, `PERCENT_RANK()` computes the relative rank of a value within the dataset, considering only the values that are strictly less than the given value, and excluding the current value itself.
- - -
 
As shown in the result set, the `PERCENT_RANK()` function assigns a percentile rank to each row based on its position within the ordered result set. This percentile rank represents the percentage of rows in the result set that are ranked lower than the current row or above the current row in the window partition.

Let's see how the `PERCENT_RANK()` function allows us to solve real-world problems.

Suppose you're a data analyst at a cell phone store and want to analyze the monthly sales performance of two phone brands, Apple and Samsung, based on their revenue quartiles. You have a table named `phone_sales_by_month`, which contains the monthly sales data by phone brand and model as follows:

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
  <th>unit_price</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>50</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>30</td>
  <td>650.00</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Galaxy S22 Ultra</td>
  <td>40</td>
  <td>799.00</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>40</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>35</td>
  <td>650.00</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Galaxy A53</td>
  <td>25</td>
  <td>415.00</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>38</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>60</td>
  <td>650.00</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>30</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Galaxy S22 Ultra</td>
  <td>25</td>
  <td>799.00</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>30</td>
  <td>650.00</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>60</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>20</td>
  <td>999.00</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Galaxy A53</td>
  <td>45</td>
  <td>415.00</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>76</td>
  <td>650.00</td>
</tr>
</table>
</body>
</html>

To analyze the monthly sales performance, you need to calculate the percentile rank of the revenue generated by each phone model and use it to determine the revenue quartiles for each model. 
The following query returns the desired results:

```sql
SELECT *,
       PERCENT_RANK() OVER(PARTITION BY brand ORDER BY quantity*unit_price DESC) AS percent_rank,
       CASE
           WHEN PERCENT_RANK() OVER(PARTITION BY brand ORDER BY quantity*unit_price DESC) < 0.25 THEN '1st'
           WHEN PERCENT_RANK() OVER(PARTITION BY brand ORDER BY quantity*unit_price DESC) < 0.50 THEN '2nd'
           WHEN PERCENT_RANK() OVER(PARTITION BY brand ORDER BY quantity*unit_price DESC) < 0.75 THEN '3rd'
           ELSE '4th'
       END AS revenue_quartiles
FROM phone_sales_by_month;
```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>model</th>
  <th>quantity</th>
  <th>unit_price</th>
  <th>percent_rank</th>
  <th>revenue_quartiles</th>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>60</td>
  <td>999.00</td>
  <td>0</td>
  <td>1st</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>50</td>
  <td>999.00</td>
  <td>0.2</td>
  <td>1st</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>40</td>
  <td>999.00</td>
  <td>0.4</td>
  <td>2nd</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>38</td>
  <td>999.00</td>
  <td>0.6</td>
  <td>3rd</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>30</td>
  <td>999.00</td>
  <td>0.8</td>
  <td>4th</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>iPhone 13 Pro</td>
  <td>20</td>
  <td>999.00</td>
  <td>1</td>
  <td>4th</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>76</td>
  <td>650.00</td>
  <td>0</td>
  <td>1st</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>60</td>
  <td>650.00</td>
  <td>0.125</td>
  <td>1st</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Galaxy S22 Ultra</td>
  <td>40</td>
  <td>799.00</td>
  <td>0.25</td>
  <td>2nd</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>35</td>
  <td>650.00</td>
  <td>0.375</td>
  <td>2nd</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>Galaxy S22 Ultra</td>
  <td>25</td>
  <td>799.00</td>
  <td>0.5</td>
  <td>3rd</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>30</td>
  <td>650.00</td>
  <td>0.625</td>
  <td>3rd</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>Galaxy Z Fold 4</td>
  <td>30</td>
  <td>650.00</td>
  <td>0.625</td>
  <td>3rd</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>Galaxy A53</td>
  <td>45</td>
  <td>415.00</td>
  <td>0.875</td>
  <td>4th</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>Galaxy A53</td>
  <td>25</td>
  <td>415.00</td>
  <td>1</td>
  <td>4th</td>
</tr>
</table>
</body>
</html>

The `CASE` statement assigns a revenue quartile to each monthly sale based on their percent rank. If their percent rank is less than 0.25, they're assigned to the first quartile; if it's less than 0.50, they're assigned to the second quartile, and so on. Any monthly sales with a percent rank greater than or equal to 0.75 are assigned to the fourth quartile.

Looking at the revenue quartiles, it's evident that the revenues generated by selling Apple phones in May and January 2022 fall into the first revenue quartile, so we can consider them the two top-performing months for Apple products. Likewise, July and March 2022 are the two top-performing months for Samsung phones. 


### Instructions
The management of the cell phone store wants to analyze the store's sales performance by identifying the top-performing cell phone models each month and returning the monthly revenue generated by those models.

A model's monthly revenue is considered top-performing if its percentile rank falls into the first revenue quartile. To fulfill the management requirement, write a query against the `phone_sales_by_month` table.

1. Create a CTE called "monthly_sales" that performs the following steps:
   * Calculate the monthly sales revenue for each cell phone model by multiplying the `quantity` and `unit_price`, then sum the result. Alias this result as `total_sales`.
   * Group the data by sales_date, brand, and model.
   * Use the `PERCENT_RANK()` window function to calculate the revenue percentile rank for each model's monthly sales.
   * Partition the data by `sales_date` and order the partitions by the calculated total sales in descending order.
2. In the main query, select the `sales_date`, `brand`, `model`, and `total_sales` from the "monthly_sales" CTE.
3. Identify the top-performing models by filtering out the rows where the total sale's percentile rank is less than or equal to 0.25.


### Hint
* Use the following statement to calculate the percentile rank of the revenue of a cell phone model in each month:

```sql
percent_rank() OVER (PARTITION BY sales_date
                     ORDER BY SUM(quantity * unit_price) DESC) AS revenue_percentile_rank
```

* Use the `GROUP BY` clause to group the results by the `sales_date`, `brand`, and `model` columns.


```sql
WITH monthly_sales AS (
SELECT sales_date, brand, model,
         SUM(quantity * unit_price) AS total_sales,
         percent_rank() OVER (PARTITION BY sales_date
                              ORDER BY SUM(quantity * unit_price) DESC) AS revenue_percentile_rank
  FROM phone_sales_by_month
 GROUP BY sales_date, brand, model
)
SELECT  sales_date, brand, model, total_sales
  FROM monthly_sales
 WHERE revenue_percentile_rank <= 0.25;
```

## The Inverse Distribution Functions - Part 1
We already discussed rank distribution functions in the previous section, which compute the relative rank of the current row in the window partition and return a ratio between 0 and 1 that can be expressed as a percentage between 0 and 100. 

Inverse distribution or percentile functions compute the inverse of rank distribution functions. Inverse distribution functions take a given percentage and return the value or an estimated (**interpolated**) value  associated with that percentage within the data range.
- - -

**NOTE**

Interpolation is a statistical method that estimates a value within a range of known values in a given dataset and inserts it between two existing values. 
- - -

Let's elaborate on the theoretical concept with an example.
Consider the following list of sorted values:
`1, 4, 5, 41, 100, 2100, 79000`
The fiftieth percentile or the *median* of the given values is 41. 
Since the set has seven values, the median falls at the fourth rank, which means that 50% of the values are below rank four, and 50% are above rank four.

What if a sorted list contains even values?
In this case, the median will be an interpolated value between the two middle values.

For example, consider the following list of sorted values:

`1, 4, 5, 41, 100, 2100`

Now we have six values, so we can't just take the value in the middle of the list as the median.

To find the median in this case, we need to take the average of the two middle values. The middle two values in this case are 5 and 41, so we add them together and divide by two to get (5 + 41) / 2 = 23. So the median of this list is 23, which is an interpolated value between the two middle values.

Let's move on from the theoretical discussion and discover how SQL helps us to perform such valuable calculations.

Before discussing the two inverse distribution functions, let's discuss a new SQL clause, `WITHIN GROUP`.

`WITHIN GROUP` is a particular clause where we can indicate the ordering expression, enabling aggregate and analytics functions to operate on a group of sorted rows rather than a group of unsorted rows created by the `GROUP BY` clause.

- - -

**NOTE**

ANSI SQL standard does not explicitly mention the `WITHIN GROUP` clause as part of the standard. However, the `WITHIN GROUP` clause is widely supported by various SQL database management systems, such as PostgreSQL, Oracle, SQL Server, and others.

- - -

Rank distribution functions are window functions that assign a different percentile rank to each row in a window partition. However, since inverse distribution functions require a specified percentile as an input, an ordering specification within a group, and returning a single result per group, they are used as grouped functions. 

The basic syntax of the `WITHIN GROUP` clause is as follows:

```sql
function_name() WITHIN GROUP (ORDER BY column_expression [ ASC | DESC ])
```


## The Inverse Distribution Functions - Part 2
There are two inverse distribution functions, `PERCENTILE_DISC()` and `PERCENTILE_CONT()`.

The `PERCENTILE_DISC()` (percentile discrete) and `PERCENTILE_CONT()` (percentile continuous) functions take a percent rank and find the value at that position within a group of rows.

According to the PostgreSQL documentation, the two functions belong to a group of functions called [Ordered-Set Aggregate Functions](https://www.postgresql.org/docs/9.4/functions-aggregate.html#FUNCTIONS-ORDEREDSET-TABLE), and the syntax is as follows:

```sql
PERCENTILE_CONT(percent) WITHIN GROUP(ORDER BY expression)
```

```sql
PERCENTILE_DISC(percent) WITHIN GROUP(ORDER BY expression)
```

Consider that the syntax of the percentile functions varies depending on which SQL flavor we're using.

- - -

**NOTE**

Although `PERCENTILE_DISC()` and `PERCENTILE_CONT()` functions are related to window functions and share some similarities, they're not considered window functions themselves. They fall under the category of ordered-set aggregate functions, which also operate on a set of rows with specified ordering, but have different syntax and usage patterns. This means that, unlike window functions, we can use these two functions in the WHERE clause of an SQL query, but we must use a subquery or a common table expression (CTE) to perform the calculation first, then reference the result in the WHERE clause.

- - -

Let's see how we use the percentile functions in practice and their differences.

We've earlier explored the `phone_sales_quantity_by_month` table containing the sold phone quantities in Q1 and Q2 2022 as follows:

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>sales_date</th>
  <th>brand</th>
  <th>quantity</th>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Apple</td>
  <td>110</td>
</tr>
<tr>
  <td>2022-01-31</td>
  <td>Samsung</td>
  <td>117</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Samsung</td>
  <td>75</td>
</tr>
<tr>
  <td>2022-02-28</td>
  <td>Apple</td>
  <td>60</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Apple</td>
  <td>85</td>
</tr>
<tr>
  <td>2022-03-31</td>
  <td>Samsung</td>
  <td>86</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Apple</td>
  <td>134</td>
</tr>
<tr>
  <td>2022-04-30</td>
  <td>Samsung</td>
  <td>124</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Samsung</td>
  <td>80</td>
</tr>
<tr>
  <td>2022-05-31</td>
  <td>Apple</td>
  <td>90</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Apple</td>
  <td>100</td>
</tr>
<tr>
  <td>2022-06-30</td>
  <td>Samsung</td>
  <td>89</td>
</tr>
</table>
</body>
</html>


Let's use the percentile functions to calculate the median of sold phone quantities.

First, we'll try the `PERCENTILE_CONT()` function.

```sql
SELECT
 PERCENTILE_CONT(0.50) WITHIN GROUP(ORDER BY quantity) as "Median of Quantity"
  FROM phone_sales_quantity_by_month;
```
<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>Median of Quantity</th>
</tr>
<tr>
  <td>89.5</td>
</tr>
</table>
</body>
</html>

Since the table contains an even number of rows (12 rows), there's no single middle value in the `quantity` column. Therefore, as its name implies, the `PERCENTILE_CONT()` function returns the interpolated value between the two values in the middle of the sorted column. This interpolated value is calculated by taking the average of those two values, resulting in 89.5.


Now, let's try the `PERCENTILE_DISC()` function.

```sql
SELECT
 PERCENTILE_DISC(0.50) WITHIN GROUP(ORDER BY quantity) as "Median of Quantity"
  FROM phone_sales_quantity_by_month;

```

<!DOCTYPE html>
<html>
  <head>
    <title></title>
    <meta charset="UTF-8">
  </head>
<body>
<table border="1" style="border-collapse:collapse">
<tr>
  <th>Median of Quantity</th>
</tr>
<tr>
  <td>89</td>
</tr>
</table>
</body>
</html>


By looking at the result we can understand, the `PERCENTILE_DISC()` function, as its name implies, returns the closest discrete value to the requested percentile. In other words, the function returns the existing value in the dataset that is immediately lower than or equal to the requested percentile.

Therefore, `PERCENTILE_DISC(0.50)` returns the closest discrete value to the actual median of the `quantity` values, which is 89. 
### Instructions
Let's assume the cell phone store wants to make informed decisions about inventory and marketing plans for next year's high-demand periods by identifying which months had high sales volumes. To do this, it's essential to identify the months when sales volumes are in the top 25% of all sales volumes.

Write a query against the `phone_sales_quantity_by_month` table to retrieve those months when the sales volume is greater or equal to the 75th percentile of quantity.

1. Use a subquery to calculate the 75th percentile value of the quantity column.
   * Utilize the `PERCENTILE_DISC()` function with a percentile value of 0.75.
   * Use the `WITHIN GROUP()` clause to sort the `quantity` values in ascending order before calculating the percentile value.
2. Select all columns from the `phone_sales_quantity_by_month` table in the main query.
3. Filter the main query results by comparing the `quantity` column with the calculated 75th percentile value, selecting rows where the quantity is greater or equal to the percentile value.

### Hint
* Write a subquery and use the statement below to retrieve the 75th percentile value of the `quantity` column.

```sql
PERCENTILE_DISC(0.75) WITHIN GROUP(ORDER BY quantity) as  "75th percentile of Quantity"
```


## Solving a Real-World Problem with the Distribution Window Functions
As we near the end of the lesson, let's practice using the distribution window functions to solve real-world problems. 

Assume the bike-sharing company will analyze the customer data to identify the most popular bike stations and the most popular routes by focusing on members. 
The result of the analysis allows the company to optimize bike availability at the popular rental stations and improve customer experience and retention.

You're asked to write an SQL query to return the most frequent start and end stations and the percentage of the rental by customer type.

As the company wants to allocate its resources on the routes where the members rent bikes more frequently, you need to identify the top first percentile of the start and end station combination where the members rent bikes.

Before writing the SQL query, it's essential to address one more clause related to window functions. Although it may be tempting to jump straight into writing code, understanding this clause will enhance your ability to use window functions effectively in real-world scenarios.

Let's explore this clause before diving into the code.

### The `WINDOW` Clause

The SQL standard `WINDOW` clause is an alternative to the `OVER()` clause which helps to write more neat queries and avoid duplication in cases where multiple window functions with similar OVER clauses are used.

The general syntax of the `WINDOW` clause is as follows:

```sql
WINDOW window_name AS 
(
	[partition_definition] 
	[order_definition] 
	[frame_definition] 
)
```
The `WINDOW` clause is placed after the `GROUP BY` and `HAVING` clauses (if they are present) and before the `ORDER BY` clause in a `SELECT` query. It's also possible to include multiple WINDOW clauses within a query.

For example, the query below returns the average salary of each department.

```sql
SELECT *,
    AVG(salary) OVER win1
  FROM employees
WINDOW win1 AS (
       PARTITION BY department
);
```

Let's get back to the task.

### Instructions
To address the above requirement, write a query against the `tbl_bikeshare` table.

1. Write a CTE to calculate the percentile rank of each start and end station combination for each member type using the `PERCENT_RANK()` window function, rounded to four decimal places. Continuing within the CTE:

* Group the rows in the table by `start_station`, `end_station`, and `member_type` using the `GROUP BY` clause, so the `COUNT()` function can be used to count the number of rentals for each combination.
* Use the `WINDOW` clause to make the query neat and easy to follow.
* Alias the new percentile rank column as `rental_percentile`.
* Order your CTE result set by `rental_percentile` in descending order.

3. Select all columns from the CTE in the main query, including the start and end stations, the member type, and the percentile rank.

4. Filter the results to include only rentals made by members and start and end station combinations with a percentile rank greater than 0.99.

### Hint
* Use the statement below to calculate the rental percentile:

```sql
ROUND(PERCENT_RANK() OVER win::numeric,4) AS rental_percentile
```
* Implement the window called `win` using the following statement:

```sql
WINDOW win AS
              (
                  PARTITION BY member_type ORDER BY COUNT(*) ASC
              )
```

```sql
WITH rental_per AS
         (
             SELECT start_station,
                     end_station,
                     member_type,
                     ROUND(PERCENT_RANK() OVER win::numeric,4) AS rental_percentile
              FROM tbl_bikeshare
              GROUP BY start_station, end_station, member_type
              WINDOW win AS
              (
                  PARTITION BY member_type ORDER BY COUNT(*)
              )
              ORDER BY rental_percentile DESC
         )
SELECT *
  FROM rental_per
 WHERE member_type = 'Member' AND rental_percentile > 0.99;
```

## Review
Congratulations on finishing this lesson!

Along the way, we've learned about the following:

* The difference between rank and inverse distribution window functions. 
* Using the `WITHIN GROUP` clause to operate on a group of sorted rows.
* The applications of the functions for performing data analysis and solving real-world problems.
* The `WINDOW` clause to write more neat queries and avoid duplication.

We're done with the SQL Window Functions course.
You can apply what you've learned in this course to leverage your advanced SQL query skills.



## Takeaways
## Syntax
* The CUME_DIST() Function:

  ```
  CUME_DIST() OVER ( [PARTITION BY expression, ... ]
                     ORDER BY expression [ASC | DESC], ... )
  ```

* The PERCENT_RANK() Function:

  ```
  PERCENT_RANK() OVER ( [PARTITION BY expression, ... ]
                        ORDER BY expression [ASC | DESC], ... )
  ```

* The PERCENTILE_CONT() Function:

  ```
  PERCENTILE_CONT(percent) WITHIN GROUP(ORDER BY expression)
  ```

* The PERCENTILE_DISC() Function:

  ```
  PERCENTILE_DISC(percent) WITHIN GROUP(ORDER BY expression)
  ```

* The WINDOW Clause:

  ```
  WINDOW window_name AS 
  (
  	[partition_definition] 
      [order_definition] 
      [frame_definition] 
  )
  ```

* the WITHIN GROUP Clause:

  ```
  function_name() WITHIN GROUP (ORDER BY column_expression [ ASC | DESC ])
  ```


## Concepts

* There are two types of distribution window functions in SQL:

1. **Rank distribution functions** are used to calculate the relative rank of a specific row in a window partition based on the ordering specified in the `OVER()` clause.

    
* `PERCENT_RANK()` calculates the relative rank of a specific row within a window partition as a percentage value between 0 and 1, with 0 representing the first row and 1 representing the last row in the partition.


* `CUME_DIST()` computes the cumulative distribution of a specific row within a window partition, which is the proportion of rows with values less than or equal to the current row's value.

        
2. **Inverse distribution functions** are used to calculate the value at a specified percentile in a group based on the ordering specified in the `WITHIN GROUP()` clause.

    
* `PERCENTILE_CONT()` is a continuous inverse distribution function that calculates the value at a specified percentile within a group using linear interpolation.


* `PERCENTILE_DISC()` is a discrete inverse distribution function that finds the value at a specified percentile within a group, returning the first value with a cumulative distribution greater than or equal to the specified percentile.  


* `WITHIN GROUP` is a clause used to specify an ordering expression for aggregate and analytic functions, allowing them to operate on sorted rows within a group instead of the unsorted rows created by the `GROUP BY` clause. It is particularly useful for inverse distribution functions, which require an ordering specification within a group and return a single result per group, such as `PERCENTILE_CONT()` and `PERCENTILE_DISC()`.

## References

* [PostgreSQL Window Functions](https://www.postgresql.org/docs/15/functions-window.html)
* [PostgreSQL Aggregate Functions](https://www.postgresql.org/docs/9.4/functions-aggregate.html)
* 

#DQ